In [1]:
!pip install requests beautifulsoup4

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import os

# CSVファイル名
filename = "seven_eleven_product.csv"

# 栄養成分を正規表現で分解する関数
def parse_nutrition(text):
    nutrition = {
        "calorie": "N/A",
        "protein": "N/A",
        "fat": "N/A",
        "carbohydrate": "N/A",
        "sugar": "N/A",
        "fiber": "N/A",
        "salt": "N/A",
    }

    # 各項目を正規表現で抽出
    patterns = {
        "calorie": r"熱量[:：]\s*([\d\.]+kcal)",
        "protein": r"たんぱく質[:：]\s*([\d\.]+g)",
        "fat": r"脂質[:：]\s*([\d\.]+g)",
        "carbohydrate": r"炭水化物[:：]\s*([\d\.]+g)",
        "sugar": r"糖質[:：]\s*([\d\.]+g)",
        "fiber": r"食物繊維[:：]\s*([\d\.]+g)",
        "salt": r"食塩相当量[:：]\s*([\d\.]+g)",
    }

    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            nutrition[key] = match.group(1)

    return nutrition


# 商品ページから詳細を取得
def scrape_seven_eleven_product(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 商品名
    product_name_div = soup.find('div', class_='item_ttl')
    product_name_tag = product_name_div.find('h1') if product_name_div else None
    product_name = product_name_tag.get_text(strip=True).replace("\u3000", "") if product_name_tag else "N/A"

    # 価格
    price_tag = soup.find('div', class_='item_price')
    price = "N/A"
    if price_tag:
        price_text = price_tag.text.strip()
        price_match = re.search(r'(\d+円（税込\d+円）)', price_text)
        price = price_match.group(1) if price_match else price_text

    # 画像URL
    picture_url = ""
    product_wrap_div = soup.find('div', class_='productWrap')
    if product_wrap_div:
        img_tag = product_wrap_div.find('img')
        if img_tag and img_tag.has_attr("src"):
            picture_url = img_tag['src']

    # 栄養成分
    nutrition_td_text = ""
    for th in soup.find_all("th"):
        if "栄養成分" in th.get_text(strip=True):
            td = th.find_next("td")
            if td:
                nutrition_td_text = td.get_text(" ", strip=True)
                break
    nutrients = parse_nutrition(nutrition_td_text)

    return {
        "product_name": product_name,
        "price": price,
        "picture": picture_url,
        "url": url,
        **nutrients
    }


# CSVに保存（重複回避して追記）
def save_to_csv(data, filename=filename):
    file_exists = os.path.isfile(filename)

    fieldnames = ["product_name", "price", "picture", "url",
                  "calorie", "protein", "fat", "carbohydrate", "sugar", "fiber", "salt"]

    with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # 新規作成時のみヘッダーを書き込む
        if not file_exists:
            writer.writeheader()

        # 既存ファイルの重複チェック
        if file_exists:
            with open(filename, 'r', encoding='utf-8') as f:
                existing = f.read()
            if data["url"] in existing:
                print(f"スキップ（既存）：{data['product_name']}")
                return

        writer.writerow(data)
        print(f"追加保存：{data['product_name']}")


# カテゴリページをクロールして商品URLを収集
def scrape_category(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    product_links = []
    for a in soup.select("div.list_inner a"):
        href = a.get("href")
        if href and "/products/" in href:
            full_url = "https://www.sej.co.jp" + href
            product_links.append(full_url)

    return list(set(product_links))  # 重複除去


# 使用例
if __name__ == "__main__":
    category_url = "https://www.sej.co.jp/products/a/onigiri"  # おにぎりカテゴリ例
    product_urls = scrape_category(category_url)

    for url in product_urls:
        try:
            product_info = scrape_seven_eleven_product(url)
            save_to_csv(product_info)
        except Exception as e:
            print(f"エラー：{url} -> {e}")


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import os

# CSVファイル名
filename = "seven_eleven_product.csv"

# 栄養成分を正規表現で分解する関数
def parse_nutrition(text):
    nutrition = {
        "calorie": "N/A",
        "protein": "N/A",
        "fat": "N/A",
        "carbohydrate": "N/A",
        "sugar": "N/A",
        "fiber": "N/A",
        "salt": "N/A",
    }

    # 各項目を正規表現で抽出
    patterns = {
        "calorie": r"熱量[:：]\s*([\d\.]+kcal)",
        "protein": r"たんぱく質[:：]\s*([\d\.]+g)",
        "fat": r"脂質[:：]\s*([\d\.]+g)",
        "carbohydrate": r"炭水化物[:：]\s*([\d\.]+g)",
        "sugar": r"糖質[:：]\s*([\d\.]+g)",
        "fiber": r"食物繊維[:：]\s*([\d\.]+g)",
        "salt": r"食塩相当量[:：]\s*([\d\.]+g)",
    }

    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            nutrition[key] = match.group(1)

    return nutrition


# 商品ページから詳細を取得
def scrape_seven_eleven_product(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 商品名
    product_name_div = soup.find('div', class_='item_ttl')
    product_name_tag = product_name_div.find('h1') if product_name_div else None
    product_name = product_name_tag.get_text(strip=True).replace("\u3000", "") if product_name_tag else "N/A"

    # 価格
    price_tag = soup.find('div', class_='item_price')
    price = "N/A"
    if price_tag:
        price_text = price_tag.text.strip()
        price_match = re.search(r'(\d+円（税込\d+円）)', price_text)
        price = price_match.group(1) if price_match else price_text

    # 画像URL
    picture_url = ""
    product_wrap_div = soup.find('div', class_='productWrap')
    if product_wrap_div:
        img_tag = product_wrap_div.find('img')
        if img_tag and img_tag.has_attr("src"):
            picture_url = img_tag['src']

    # 栄養成分
    nutrition_td_text = ""
    for th in soup.find_all("th"):
        if "栄養成分" in th.get_text(strip=True):
            td = th.find_next("td")
            if td:
                nutrition_td_text = td.get_text(" ", strip=True)
                break
    nutrients = parse_nutrition(nutrition_td_text)

    return {
        "product_name": product_name,
        "price": price,
        "picture": picture_url,
        "url": url,
        **nutrients
    }


# CSVに保存（重複回避して追記）
def save_to_csv(data, filename=filename):
    file_exists = os.path.isfile(filename)

    fieldnames = ["product_name", "price", "picture", "url",
                  "calorie", "protein", "fat", "carbohydrate", "sugar", "fiber", "salt"]

    with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # 新規作成時のみヘッダーを書き込む
        if not file_exists:
            writer.writeheader()

        # 既存ファイルの重複チェック
        if file_exists:
            with open(filename, 'r', encoding='utf-8') as f:
                existing = f.read()
            if data["url"] in existing:
                print(f"スキップ（既存）：{data['product_name']}")
                return

        writer.writerow(data)
        print(f"追加保存：{data['product_name']}")


# カテゴリページをクロールして商品URLを収集
def scrape_category(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    product_links = []
    for a in soup.select("div.list_inner a"):
        href = a.get("href")
        if href and "/products/" in href:
            full_url = "https://www.sej.co.jp" + href
            product_links.append(full_url)

    return list(set(product_links))  # 重複除去


# 使用例
if __name__ == "__main__":
    category_url = "https://www.sej.co.jp/products/a/sushi/"  # おにぎりカテゴリ例
    product_urls = scrape_category(category_url)

    for url in product_urls:
        try:
            product_info = scrape_seven_eleven_product(url)
            save_to_csv(product_info)
        except Exception as e:
            print(f"エラー：{url} -> {e}")


エラー：https://www.sej.co.jp/products/a/item/044389/ -> dict contains fields not in fieldnames: 'category'
エラー：https://www.sej.co.jp/products/a/item/047652/ -> dict contains fields not in fieldnames: 'category'
エラー：https://www.sej.co.jp/products/a/item/047807/ -> dict contains fields not in fieldnames: 'category'
エラー：https://www.sej.co.jp/products/a/item/046758/ -> dict contains fields not in fieldnames: 'category'
エラー：https://www.sej.co.jp/products/a/item/047767/ -> dict contains fields not in fieldnames: 'category'
エラー：https://www.sej.co.jp/products/a/item/047562/ -> dict contains fields not in fieldnames: 'category'
エラー：https://www.sej.co.jp/products/a/item/047808/ -> dict contains fields not in fieldnames: 'category'
エラー：https://www.sej.co.jp/products/a/item/047611/ -> dict contains fields not in fieldnames: 'category'
エラー：https://www.sej.co.jp/products/a/item/047765/ -> dict contains fields not in fieldnames: 'category'
エラー：https://www.sej.co.jp/products/a/item/047576/ -> dict contai

KeyboardInterrupt: 

In [5]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import os

# CSVファイル名
filename = "seven_eleven_product.csv"

# 栄養成分を正規表現で分解する関数
def parse_nutrition(text):
    nutrition = {
        "calorie": "N/A",
        "protein": "N/A",
        "fat": "N/A",
        "carbohydrate": "N/A",
        "sugar": "N/A",
        "fiber": "N/A",
        "salt": "N/A",
    }

    # 各項目を正規表現で抽出
    patterns = {
        "calorie": r"熱量[:：]\s*([\d\.]+kcal)",
        "protein": r"たんぱく質[:：]\s*([\d\.]+g)",
        "fat": r"脂質[:：]\s*([\d\.]+g)",
        "carbohydrate": r"炭水化物[:：]\s*([\d\.]+g)",
        "sugar": r"糖質[:：]\s*([\d\.]+g)",
        "fiber": r"食物繊維[:：]\s*([\d\.]+g)",
        "salt": r"食塩相当量[:：]\s*([\d\.]+g)",
    }

    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            nutrition[key] = match.group(1)

    return nutrition


# 商品ページから詳細を取得
def scrape_seven_eleven_product(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 商品名
    product_name_div = soup.find('div', class_='item_ttl')
    product_name_tag = product_name_div.find('h1') if product_name_div else None
    product_name = product_name_tag.get_text(strip=True).replace("\u3000", "") if product_name_tag else "N/A"

    # 価格
    price_tag = soup.find('div', class_='item_price')
    price = "N/A"
    if price_tag:
        price_text = price_tag.text.strip()
        price_match = re.search(r'(\d+円（税込\d+円）)', price_text)
        price = price_match.group(1) if price_match else price_text

    # 画像URL
    picture_url = ""
    product_wrap_div = soup.find('div', class_='productWrap')
    if product_wrap_div:
        img_tag = product_wrap_div.find('img')
        if img_tag and img_tag.has_attr("src"):
            picture_url = img_tag['src']

    # 栄養成分
    nutrition_td_text = ""
    for th in soup.find_all("th"):
        if "栄養成分" in th.get_text(strip=True):
            td = th.find_next("td")
            if td:
                nutrition_td_text = td.get_text(" ", strip=True)
                break
    nutrients = parse_nutrition(nutrition_td_text)

    return {
        "product_name": product_name,
        "price": price,
        "picture": picture_url,
        "url": url,
        **nutrients
    }


# CSVに保存（重複回避して追記）
def save_to_csv(data, filename=filename):
    file_exists = os.path.isfile(filename)

    fieldnames = ["product_name", "price", "picture", "url",
                  "calorie", "protein", "fat", "carbohydrate", "sugar", "fiber", "salt"]

    with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # 新規作成時のみヘッダーを書き込む
        if not file_exists:
            writer.writeheader()

        # 既存ファイルの重複チェック
        if file_exists:
            with open(filename, 'r', encoding='utf-8') as f:
                existing = f.read()
            if data["url"] in existing:
                print(f"スキップ（既存）：{data['product_name']}")
                return

        writer.writerow(data)
        print(f"追加保存：{data['product_name']}")


# カテゴリページをクロールして商品URLを収集
def scrape_category(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    product_links = []
    for a in soup.select("div.list_inner a"):
        href = a.get("href")
        if href and "/products/" in href:
            full_url = "https://www.sej.co.jp" + href
            product_links.append(full_url)

    return list(set(product_links))  # 重複除去


# 使用例
if __name__ == "__main__":
    category_url = "https://www.sej.co.jp/products/a/bento/"  # おにぎりカテゴリ例
    product_urls = scrape_category(category_url)

    for url in product_urls:
        try:
            product_info = scrape_seven_eleven_product(url)
            save_to_csv(product_info)
        except Exception as e:
            print(f"エラー：{url} -> {e}")


追加保存：チキンたれかつ弁当
追加保存：東海限定特製天津飯
追加保存：埼玉ゆかりの味かてめし（混ぜご飯）
追加保存：秋の味覚北海道産秋鮭ほぐしのおだしごはん
追加保存：香味だれで食べる揚げナスと豚しゃぶ丼
追加保存：黒胡椒仕立てのかしわバター丼
追加保存：秋の味覚牛肉きのこ御飯幕の内
追加保存：背徳飯やみつきタルだく唐揚げ丼
追加保存：旨辛チーズタッカルビ丼
追加保存：コク旨だれの炭火焼き牛カルビ弁当
追加保存：ピリ辛もつ炒め丼
追加保存：若鶏のにんにく醤油唐揚げ弁当


In [14]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import os

# CSVファイル名
filename = "seven_eleven_product.csv"

# 栄養成分を正規表現で分解する関数
def parse_nutrition(text):
    nutrition = {
        "calorie": "N/A",
        "protein": "N/A",
        "fat": "N/A",
        "carbohydrate": "N/A",
        "sugar": "N/A",
        "fiber": "N/A",
        "salt": "N/A",
    }

    # 各項目を正規表現で抽出
    patterns = {
        "calorie": r"熱量[:：]\s*([\d\.]+kcal)",
        "protein": r"たんぱく質[:：]\s*([\d\.]+g)",
        "fat": r"脂質[:：]\s*([\d\.]+g)",
        "carbohydrate": r"炭水化物[:：]\s*([\d\.]+g)",
        "sugar": r"糖質[:：]\s*([\d\.]+g)",
        "fiber": r"食物繊維[:：]\s*([\d\.]+g)",
        "salt": r"食塩相当量[:：]\s*([\d\.]+g)",
    }

    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            nutrition[key] = match.group(1)

    return nutrition


# 商品ページから詳細を取得
def scrape_seven_eleven_product(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 商品名
    product_name_div = soup.find('div', class_='item_ttl')
    product_name_tag = product_name_div.find('h1') if product_name_div else None
    product_name = product_name_tag.get_text(strip=True).replace("\u3000", "") if product_name_tag else "N/A"

    # 価格
    price_tag = soup.find('div', class_='item_price')
    price = "N/A"
    if price_tag:
        price_text = price_tag.text.strip()
        price_match = re.search(r'(\d+円（税込\d+円）)', price_text)
        price = price_match.group(1) if price_match else price_text

    # 画像URL
    picture_url = ""
    product_wrap_div = soup.find('div', class_='productWrap')
    if product_wrap_div:
        img_tag = product_wrap_div.find('img')
        if img_tag and img_tag.has_attr("src"):
            picture_url = img_tag['src']

    # 栄養成分
    nutrition_td_text = ""
    for th in soup.find_all("th"):
        if "栄養成分" in th.get_text(strip=True):
            td = th.find_next("td")
            if td:
                nutrition_td_text = td.get_text(" ", strip=True)
                break
    nutrients = parse_nutrition(nutrition_td_text)

    return {
        "product_name": product_name,
        "price": price,
        "picture": picture_url,
        "url": url,
        **nutrients
    }


# CSVに保存（重複回避して追記）
def save_to_csv(data, filename=filename):
    file_exists = os.path.isfile(filename)

    fieldnames = ["product_name", "price", "picture", "url",
                  "calorie", "protein", "fat", "carbohydrate", "sugar", "fiber", "salt"]

    with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # 新規作成時のみヘッダーを書き込む
        if not file_exists:
            writer.writeheader()

        # 既存ファイルの重複チェック
        if file_exists:
            with open(filename, 'r', encoding='utf-8') as f:
                existing = f.read()
            if data["url"] in existing:
                print(f"スキップ（既存）：{data['product_name']}")
                return

        writer.writerow(data)
        print(f"追加保存：{data['product_name']}")


# # カテゴリページをクロールして商品URLを収集
# def scrape_category(url):
#     response = requests.get(url)
#     soup = BeautifulSoup(response.content, 'html.parser')

#     product_links = []
#     for a in soup.select("div.list_inner a"):
#         href = a.get("href")
#         if href and "/products/" in href:
#             full_url = "https://www.sej.co.jp" + href
#             product_links.append(full_url)

#     return list(set(product_links))  # 重複除去

from urllib.parse import urljoin

def scrape_category_all_pages(base_url):
    all_links = set()
    visited_pages = set()
    next_pages = [base_url]

    while next_pages:
        url = next_pages.pop(0)
        if url in visited_pages:
            continue
        visited_pages.add(url)

        print(f"取得中: {url}")
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        # 商品リンク収集
        for a in soup.select("div.list_inner a"):
            href = a.get("href")
            if href and "/products/" in href:
                full_url = urljoin("https://www.sej.co.jp", href)
                all_links.add(full_url)

        # ページャーから次ページ候補を収集
        pager = soup.select("div.pager a")
        for a in pager:
            href = a.get("href")
            if href:
                next_url = urljoin("https://www.sej.co.jp", href)
                if next_url not in visited_pages:
                    next_pages.append(next_url)

    return list(all_links)



# 使用例
if __name__ == "__main__":
    category_url = "https://www.sej.co.jp/products/a/donut/"
    product_urls = scrape_category_all_pages(category_url)

    print(f"商品数: {len(product_urls)}")
    for url in product_urls:
        try:
            product_info = scrape_seven_eleven_product(url)
            save_to_csv(product_info)
        except Exception as e:
            print(f"エラー：{url} -> {e}")


取得中: https://www.sej.co.jp/products/a/donut/
取得中: https://www.sej.co.jp/products/a/donut/2/l15/
取得中: https://www.sej.co.jp/products/a/donut/1/l15/
商品数: 16
追加保存：オールドファッション
追加保存：こしあんドーナツ
追加保存：チョコオールドファッション
追加保存：ふわふわ食感シフォンケーキ
追加保存：もちもちリングドーナツ
追加保存：もちもちリングドーナツチョコ
追加保存：もちもちリングシュガー
追加保存：アーモンドチョコケーキ
追加保存：もちもち食感リングドーナツチョコ
追加保存：アソートドーナツ３個入り
追加保存：富士山ミルク使用ひとくちドーナツ９個入
追加保存：大学芋みたいなオールドファッション
追加保存：ハニーオールドファッション
追加保存：チョコケーキドーナツ
追加保存：チョコオールドファッション
追加保存：しっとり黒糖ボールドーナツ２個入り


In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import os

# CSVファイル名
filename = "seven_eleven_product.csv"

# 栄養成分を正規表現で分解する関数
def parse_nutrition(text):
    nutrition = {
        "calorie": "N/A",
        "protein": "N/A",
        "fat": "N/A",
        "carbohydrate": "N/A",
        "sugar": "N/A",
        "fiber": "N/A",
        "salt": "N/A",
    }

    # 各項目を正規表現で抽出
    patterns = {
        "calorie": r"熱量[:：]\s*([\d\.]+kcal)",
        "protein": r"たんぱく質[:：]\s*([\d\.]+g)",
        "fat": r"脂質[:：]\s*([\d\.]+g)",
        "carbohydrate": r"炭水化物[:：]\s*([\d\.]+g)",
        "sugar": r"糖質[:：]\s*([\d\.]+g)",
        "fiber": r"食物繊維[:：]\s*([\d\.]+g)",
        "salt": r"食塩相当量[:：]\s*([\d\.]+g)",
    }

    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            nutrition[key] = match.group(1)

    return nutrition


# 商品ページから詳細を取得
def scrape_seven_eleven_product(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 商品名
    product_name_div = soup.find('div', class_='item_ttl')
    product_name_tag = product_name_div.find('h1') if product_name_div else None
    product_name = product_name_tag.get_text(strip=True).replace("\u3000", "") if product_name_tag else "N/A"

    # 価格
    price_tag = soup.find('div', class_='item_price')
    price = "N/A"
    if price_tag:
        price_text = price_tag.text.strip()
        price_match = re.search(r'(\d+円（税込\d+円）)', price_text)
        price = price_match.group(1) if price_match else price_text

    # 画像URL
    picture_url = ""
    product_wrap_div = soup.find('div', class_='productWrap')
    if product_wrap_div:
        img_tag = product_wrap_div.find('img')
        if img_tag and img_tag.has_attr("src"):
            picture_url = img_tag['src']

    # 栄養成分
    nutrition_td_text = ""
    for th in soup.find_all("th"):
        if "栄養成分" in th.get_text(strip=True):
            td = th.find_next("td")
            if td:
                nutrition_td_text = td.get_text(" ", strip=True)
                break
    nutrients = parse_nutrition(nutrition_td_text)

    return {
        "product_name": product_name,
        "price": price,
        "picture": picture_url,
        "url": url,
        **nutrients
    }


# CSVに保存（重複回避して追記）
def save_to_csv(data, filename=filename):
    file_exists = os.path.isfile(filename)

    fieldnames = ["product_name", "price", "picture", "url",
                  "calorie", "protein", "fat", "carbohydrate", "sugar", "fiber", "salt"]

    with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # 新規作成時のみヘッダーを書き込む
        if not file_exists:
            writer.writeheader()

        # 既存ファイルの重複チェック
        if file_exists:
            with open(filename, 'r', encoding='utf-8') as f:
                existing = f.read()
            if data["url"] in existing:
                print(f"スキップ（既存）：{data['product_name']}")
                return

        writer.writerow(data)
        print(f"追加保存：{data['product_name']}")


# # カテゴリページをクロールして商品URLを収集
# def scrape_category(url):
#     response = requests.get(url)
#     soup = BeautifulSoup(response.content, 'html.parser')

#     product_links = []
#     for a in soup.select("div.list_inner a"):
#         href = a.get("href")
#         if href and "/products/" in href:
#             full_url = "https://www.sej.co.jp" + href
#             product_links.append(full_url)

#     return list(set(product_links))  # 重複除去

# カテゴリページをクロールして全ページの商品URLを収集
def scrape_category_all_pages(base_url):
    all_links = []
    page = 1

    while True:
        # 1ページ目はそのまま、それ以降は /{page}/15 を付与
        if page == 1:
            url = base_url
        else:
            if not base_url.endswith("/"):
                base_url += "/"
            url = f"{base_url}{page+1}/15"

        print(f"取得中: {url}")
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        product_links = []
        for a in soup.select("div.list_inner a"):
            href = a.get("href")
            if href and "/products/" in href:
                full_url = "https://www.sej.co.jp" + href
                product_links.append(full_url)

        if not product_links:  # 商品が見つからなければ終了
            break

        all_links.extend(product_links)
        page += 1

    return list(set(all_links))  # 重複除去


# 使用例
if __name__ == "__main__":
    category_url = "https://www.sej.co.jp/products/a/bread/"  # おにぎりカテゴリ例
    product_urls = scrape_category(category_url)

    for url in product_urls:
        try:
            product_info = scrape_seven_eleven_product(url)
            save_to_csv(product_info)
        except Exception as e:
            print(f"エラー：{url} -> {e}")


追加保存：揚げパンシュガー
追加保存：７プレミアムふわもちブレッド６枚入
追加保存：７プレミアムもっちり食パン６枚入
追加保存：７ＰＧ金の食パン２枚入
追加保存：タルタルソースのフィッシュバーガー
追加保存：北海道産ポテトとチーズのパン
追加保存：粗挽きポークフランク粒マスタードマヨ使用
追加保存：ランチパック深煎りピーナッツ
追加保存：７Ｐマーガリン入りレーズンバターロール
追加保存：７プレミアム発酵バター香るミニクロワッサン５個入
追加保存：北海道産じゃがいものコロッケパン
追加保存：北海道産じゃがいものコロッケパン
追加保存：モリヤマメロンのクリームサンド
追加保存：北海道産大豆のきなこあげぱん
追加保存：７プレミアムマーラーカオ４個入
追加保存：チョコ棒
追加保存：７Ｐマーガリン入りバターロール４個入
追加保存：７ＰＧ金の食パン４枚入
追加保存：パスコ超熟６枚スライス
追加保存：７Ｐマーガリン入り黒糖ロール４個入
追加保存：７Ｐトリュフマヨツイスト
追加保存：たっぷりルウの欧風カレ―パン
追加保存：マロンクリームデニッシュ
追加保存：７ＰＧ金の食パン４枚


In [15]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import os

# CSVファイル名
filename = "seven_eleven_product.csv"

# 栄養成分を正規表現で分解する関数
def parse_nutrition(text):
    nutrition = {
        "calorie": "N/A",
        "protein": "N/A",
        "fat": "N/A",
        "carbohydrate": "N/A",
        "sugar": "N/A",
        "fiber": "N/A",
        "salt": "N/A",
    }

    # 各項目を正規表現で抽出
    patterns = {
        "calorie": r"熱量[:：]\s*([\d\.]+kcal)",
        "protein": r"たんぱく質[:：]\s*([\d\.]+g)",
        "fat": r"脂質[:：]\s*([\d\.]+g)",
        "carbohydrate": r"炭水化物[:：]\s*([\d\.]+g)",
        "sugar": r"糖質[:：]\s*([\d\.]+g)",
        "fiber": r"食物繊維[:：]\s*([\d\.]+g)",
        "salt": r"食塩相当量[:：]\s*([\d\.]+g)",
    }

    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            nutrition[key] = match.group(1)

    return nutrition


# 商品ページから詳細を取得
def scrape_seven_eleven_product(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 商品名
    product_name_div = soup.find('div', class_='item_ttl')
    product_name_tag = product_name_div.find('h1') if product_name_div else None
    product_name = product_name_tag.get_text(strip=True).replace("\u3000", "") if product_name_tag else "N/A"

    # 価格
    price_tag = soup.find('div', class_='item_price')
    price = "N/A"
    if price_tag:
        price_text = price_tag.text.strip()
        price_match = re.search(r'(\d+円（税込\d+円）)', price_text)
        price = price_match.group(1) if price_match else price_text

    # 画像URL
    picture_url = ""
    product_wrap_div = soup.find('div', class_='productWrap')
    if product_wrap_div:
        img_tag = product_wrap_div.find('img')
        if img_tag and img_tag.has_attr("src"):
            picture_url = img_tag['src']

    # 栄養成分
    nutrition_td_text = ""
    for th in soup.find_all("th"):
        if "栄養成分" in th.get_text(strip=True):
            td = th.find_next("td")
            if td:
                nutrition_td_text = td.get_text(" ", strip=True)
                break
    nutrients = parse_nutrition(nutrition_td_text)

    return {
        "product_name": product_name,
        "price": price,
        "picture": picture_url,
        "url": url,
        **nutrients
    }


# CSVに保存（重複回避して追記）
def save_to_csv(data, filename=filename):
    file_exists = os.path.isfile(filename)

    fieldnames = ["product_name", "price", "picture", "url",
                  "calorie", "protein", "fat", "carbohydrate", "sugar", "fiber", "salt"]

    with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # 新規作成時のみヘッダーを書き込む
        if not file_exists:
            writer.writeheader()

        # 既存ファイルの重複チェック
        if file_exists:
            with open(filename, 'r', encoding='utf-8') as f:
                existing = f.read()
            if data["url"] in existing:
                print(f"スキップ（既存）：{data['product_name']}")
                return

        writer.writerow(data)
        print(f"追加保存：{data['product_name']}")


# # カテゴリページをクロールして商品URLを収集
# def scrape_category(url):
#     response = requests.get(url)
#     soup = BeautifulSoup(response.content, 'html.parser')

#     product_links = []
#     for a in soup.select("div.list_inner a"):
#         href = a.get("href")
#         if href and "/products/" in href:
#             full_url = "https://www.sej.co.jp" + href
#             product_links.append(full_url)

#     return list(set(product_links))  # 重複除去

from urllib.parse import urljoin

def scrape_category_all_pages(base_url):
    all_links = set()
    visited_pages = set()
    next_pages = [base_url]

    while next_pages:
        url = next_pages.pop(0)
        if url in visited_pages:
            continue
        visited_pages.add(url)

        print(f"取得中: {url}")
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        # 商品リンク収集
        for a in soup.select("div.list_inner a"):
            href = a.get("href")
            if href and "/products/" in href:
                full_url = urljoin("https://www.sej.co.jp", href)
                all_links.add(full_url)

        # ページャーから次ページ候補を収集
        pager = soup.select("div.pager a")
        for a in pager:
            href = a.get("href")
            if href:
                next_url = urljoin("https://www.sej.co.jp", href)
                if next_url not in visited_pages:
                    next_pages.append(next_url)

    return list(all_links)



# 使用例
if __name__ == "__main__":
    category_url = "https://www.sej.co.jp/products/a/sandwich/"
    product_urls = scrape_category_all_pages(category_url)

    print(f"商品数: {len(product_urls)}")
    for url in product_urls:
        try:
            product_info = scrape_seven_eleven_product(url)
            save_to_csv(product_info)
        except Exception as e:
            print(f"エラー：{url} -> {e}")


取得中: https://www.sej.co.jp/products/a/sandwich/
取得中: https://www.sej.co.jp/products/a/sandwich/2/l15/
取得中: https://www.sej.co.jp/products/a/sandwich/3/l15/
取得中: https://www.sej.co.jp/products/a/sandwich/4/l15/
取得中: https://www.sej.co.jp/products/a/sandwich/5/l15/
取得中: https://www.sej.co.jp/products/a/sandwich/6/l15/
取得中: https://www.sej.co.jp/products/a/sandwich/7/l15/
取得中: https://www.sej.co.jp/products/a/sandwich/8/l15/
取得中: https://www.sej.co.jp/products/a/sandwich/9/l15/
取得中: https://www.sej.co.jp/products/a/sandwich/1/l15/
商品数: 125
追加保存：野菜ミックスサンド
追加保存：クロワッサンサンド照焼チキン＆たまご
スキップ（既存）：半熟卵てりやきバーガー
スキップ（既存）：シャインマスカット入りフルーツミックスサンド
追加保存：バゲットサンドパストラミ＆クリームチーズ
追加保存：野菜ミックスサンド
スキップ（既存）：ローストチキンと野菜のバインミー
追加保存：チーズバーガー
追加保存：たまごサンド
追加保存：ひんやりとろけるフレンチトースト
スキップ（既存）：ＢＩＧハムとたまご
追加保存：チーズバーガー
追加保存：トーストサンドハム＆スクランブルエッグ
追加保存：ラップデリスパイシーコブサラダ
追加保存：半熟卵てりやきバーガー
追加保存：クロックマダム
追加保存：ミニロール（たまご＆ハム）
追加保存：彩り野菜ミックスサンド
追加保存：嬬恋村産キャベツ使用コールスローサンド
追加保存：たんぱく質が摂れるチキン＆チリ
追加保存：九州産華味鳥チキンカツサンド
追加保存：４種のフルーツミックスサンド
追加保存：ミックスサンド
追加保存：トース

In [16]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import os

# CSVファイル名
filename = "seven_eleven_product.csv"

# 栄養成分を正規表現で分解する関数
def parse_nutrition(text):
    nutrition = {
        "calorie": "N/A",
        "protein": "N/A",
        "fat": "N/A",
        "carbohydrate": "N/A",
        "sugar": "N/A",
        "fiber": "N/A",
        "salt": "N/A",
    }

    # 各項目を正規表現で抽出
    patterns = {
        "calorie": r"熱量[:：]\s*([\d\.]+kcal)",
        "protein": r"たんぱく質[:：]\s*([\d\.]+g)",
        "fat": r"脂質[:：]\s*([\d\.]+g)",
        "carbohydrate": r"炭水化物[:：]\s*([\d\.]+g)",
        "sugar": r"糖質[:：]\s*([\d\.]+g)",
        "fiber": r"食物繊維[:：]\s*([\d\.]+g)",
        "salt": r"食塩相当量[:：]\s*([\d\.]+g)",
    }

    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            nutrition[key] = match.group(1)

    return nutrition


# 商品ページから詳細を取得
def scrape_seven_eleven_product(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 商品名
    product_name_div = soup.find('div', class_='item_ttl')
    product_name_tag = product_name_div.find('h1') if product_name_div else None
    product_name = product_name_tag.get_text(strip=True).replace("\u3000", "") if product_name_tag else "N/A"

    # 価格
    price_tag = soup.find('div', class_='item_price')
    price = "N/A"
    if price_tag:
        price_text = price_tag.text.strip()
        price_match = re.search(r'(\d+円（税込\d+円）)', price_text)
        price = price_match.group(1) if price_match else price_text

    # 画像URL
    picture_url = ""
    product_wrap_div = soup.find('div', class_='productWrap')
    if product_wrap_div:
        img_tag = product_wrap_div.find('img')
        if img_tag and img_tag.has_attr("src"):
            picture_url = img_tag['src']

    # 栄養成分
    nutrition_td_text = ""
    for th in soup.find_all("th"):
        if "栄養成分" in th.get_text(strip=True):
            td = th.find_next("td")
            if td:
                nutrition_td_text = td.get_text(" ", strip=True)
                break
    nutrients = parse_nutrition(nutrition_td_text)

    return {
        "product_name": product_name,
        "price": price,
        "picture": picture_url,
        "url": url,
        **nutrients
    }


# CSVに保存（重複回避して追記）
def save_to_csv(data, filename=filename):
    file_exists = os.path.isfile(filename)

    fieldnames = ["product_name", "price", "picture", "url",
                  "calorie", "protein", "fat", "carbohydrate", "sugar", "fiber", "salt"]

    with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # 新規作成時のみヘッダーを書き込む
        if not file_exists:
            writer.writeheader()

        # 既存ファイルの重複チェック
        if file_exists:
            with open(filename, 'r', encoding='utf-8') as f:
                existing = f.read()
            if data["url"] in existing:
                print(f"スキップ（既存）：{data['product_name']}")
                return

        writer.writerow(data)
        print(f"追加保存：{data['product_name']}")


from urllib.parse import urljoin

def scrape_category_all_pages(base_url):
    all_links = set()
    visited_pages = set()
    next_pages = [base_url]

    while next_pages:
        url = next_pages.pop(0)
        if url in visited_pages:
            continue
        visited_pages.add(url)

        print(f"取得中: {url}")
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        # 商品リンク収集
        for a in soup.select("div.list_inner a"):
            href = a.get("href")
            if href and "/products/" in href:
                full_url = urljoin("https://www.sej.co.jp", href)
                all_links.add(full_url)

        # ページャーから次ページ候補を収集
        pager = soup.select("div.pager a")
        for a in pager:
            href = a.get("href")
            if href:
                next_url = urljoin("https://www.sej.co.jp", href)
                if next_url not in visited_pages:
                    next_pages.append(next_url)

    return list(all_links)



# 使用例
if __name__ == "__main__":
    category_url = "https://www.sej.co.jp/products/a/men/"
    product_urls = scrape_category_all_pages(category_url)

    print(f"商品数: {len(product_urls)}")
    for url in product_urls:
        try:
            product_info = scrape_seven_eleven_product(url)
            save_to_csv(product_info)
        except Exception as e:
            print(f"エラー：{url} -> {e}")


取得中: https://www.sej.co.jp/products/a/men/
商品数: 24
追加保存：お月見冷しちく玉天うどん
追加保存：東北限定若鶏の冷たい肉そば
追加保存：野菜たっぷりあんかけ焼そば
追加保存：高島とんちゃん焼うどん鶏の味噌焼き
追加保存：ぼっかけ焼そば
追加保存：冷し中華
追加保存：薬味で味わう冷し月見半熟玉子うどん
追加保存：１／２日分の野菜が摂れる和風ちゃんぽん
追加保存：だしの旨みと醤油香る焼うどん
追加保存：冷し中華（マヨ付）
追加保存：岐阜の味冷したぬきそば
追加保存：信州麺友会公認ねぎと胡椒の王様中華そば
追加保存：冷しぶっかけうどん
追加保存：ミニ温そうめん上州地粉そうめん使用
追加保存：冷しぶっかけうどん
追加保存：だし割りななこいもとろろの田舎そば
追加保存：ソース焼そば
追加保存：ちゅるもち鮭ときのこのクリームうどん
追加保存：半熟味付たまごの冷製中華そば
追加保存：静岡限定のりおろしそば
追加保存：濃厚だし割りとろろの冷しぶっかけそば
追加保存：ミニ冷し中華（マヨ付）
追加保存：濃厚だし割りとろろのミニ冷しぶっかけそば
追加保存：お月見明太半熟玉子うどん


In [8]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import os

# CSVファイル名
filename = "seven_eleven_product.csv"

# 栄養成分を正規表現で分解する関数
def parse_nutrition(text):
    nutrition = {
        "calorie": "N/A",
        "protein": "N/A",
        "fat": "N/A",
        "carbohydrate": "N/A",
        "sugar": "N/A",
        "fiber": "N/A",
        "salt": "N/A",
    }

    # 各項目を正規表現で抽出
    patterns = {
        "calorie": r"熱量[:：]\s*([\d\.]+kcal)",
        "protein": r"たんぱく質[:：]\s*([\d\.]+g)",
        "fat": r"脂質[:：]\s*([\d\.]+g)",
        "carbohydrate": r"炭水化物[:：]\s*([\d\.]+g)",
        "sugar": r"糖質[:：]\s*([\d\.]+g)",
        "fiber": r"食物繊維[:：]\s*([\d\.]+g)",
        "salt": r"食塩相当量[:：]\s*([\d\.]+g)",
    }

    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            nutrition[key] = match.group(1)

    return nutrition


# 商品ページから詳細を取得
def scrape_seven_eleven_product(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 商品名
    product_name_div = soup.find('div', class_='item_ttl')
    product_name_tag = product_name_div.find('h1') if product_name_div else None
    product_name = product_name_tag.get_text(strip=True).replace("\u3000", "") if product_name_tag else "N/A"

    # 価格
    price_tag = soup.find('div', class_='item_price')
    price = "N/A"
    if price_tag:
        price_text = price_tag.text.strip()
        price_match = re.search(r'(\d+円（税込\d+円）)', price_text)
        price = price_match.group(1) if price_match else price_text

    # 画像URL
    picture_url = ""
    product_wrap_div = soup.find('div', class_='productWrap')
    if product_wrap_div:
        img_tag = product_wrap_div.find('img')
        if img_tag and img_tag.has_attr("src"):
            picture_url = img_tag['src']

    # 栄養成分
    nutrition_td_text = ""
    for th in soup.find_all("th"):
        if "栄養成分" in th.get_text(strip=True):
            td = th.find_next("td")
            if td:
                nutrition_td_text = td.get_text(" ", strip=True)
                break
    nutrients = parse_nutrition(nutrition_td_text)

    return {
        "product_name": product_name,
        "price": price,
        "picture": picture_url,
        "url": url,
        **nutrients
    }


# CSVに保存（重複回避して追記）
def save_to_csv(data, filename=filename):
    file_exists = os.path.isfile(filename)

    fieldnames = ["product_name", "price", "picture", "url",
                  "calorie", "protein", "fat", "carbohydrate", "sugar", "fiber", "salt"]

    with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # 新規作成時のみヘッダーを書き込む
        if not file_exists:
            writer.writeheader()

        # 既存ファイルの重複チェック
        if file_exists:
            with open(filename, 'r', encoding='utf-8') as f:
                existing = f.read()
            if data["url"] in existing:
                print(f"スキップ（既存）：{data['product_name']}")
                return

        writer.writerow(data)
        print(f"追加保存：{data['product_name']}")


from urllib.parse import urljoin

def scrape_category_all_pages(base_url):
    all_links = set()
    visited_pages = set()
    next_pages = [base_url]

    while next_pages:
        url = next_pages.pop(0)
        if url in visited_pages:
            continue
        visited_pages.add(url)

        print(f"取得中: {url}")
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        # 商品リンク収集
        for a in soup.select("div.list_inner a"):
            href = a.get("href")
            if href and "/products/" in href:
                full_url = urljoin("https://www.sej.co.jp", href)
                all_links.add(full_url)

        # ページャーから次ページ候補を収集
        pager = soup.select("div.pager a")
        for a in pager:
            href = a.get("href")
            if href:
                next_url = urljoin("https://www.sej.co.jp", href)
                if next_url not in visited_pages:
                    next_pages.append(next_url)

    return list(all_links)



# 使用例
if __name__ == "__main__":
    category_url = "https://www.sej.co.jp/products/a/pasta/"
    product_urls = scrape_category_all_pages(category_url)

    print(f"商品数: {len(product_urls)}")
    for url in product_urls:
        try:
            product_info = scrape_seven_eleven_product(url)
            save_to_csv(product_info)
        except Exception as e:
            print(f"エラー：{url} -> {e}")


取得中: https://www.sej.co.jp/products/a/pasta/
取得中: https://www.sej.co.jp/products/a/pasta/2/l15/
取得中: https://www.sej.co.jp/products/a/pasta/3/l15/
取得中: https://www.sej.co.jp/products/a/pasta/1/l15/
商品数: 44
追加保存：豚しゃぶと胡麻ぽん酢の和風おろし冷製パスタ
追加保存：道東限定ミートスパカツ
追加保存：きのこと鶏肉のクリームパスタ
追加保存：つぶつぶ生たらこをのせた和風パスタ
追加保存：大盛明太マヨのスパゲティ
追加保存：静岡限定和風冷製パスタツナと大根おろし
追加保存：冷製パスタ海老のトマトクリーム
追加保存：魚介の旨塩大盛パスタ
追加保存：チキンステーキのせバター醤油パスタ
追加保存：ボンゴレロッソあさりのトマトソース
追加保存：よくばりパスタミートボールのせトマトソース
追加保存：大盛明太マヨのスパゲティ
追加保存：冷製トマトソーストマトと蒸し鶏のパスタ
追加保存：麺大盛ソーセージのペペロンチーノ
追加保存：ツナと大根おろしの和風パスタ
追加保存：大盛海老といかの旨塩パスタ
追加保存：道東限定塩味スパゲティ
追加保存：海老といかの旨塩パスタ
追加保存：大盛明太マヨのスパゲティ
追加保存：アスパラとベーコンのバター醤油パスタ
追加保存：冷製パスタ豚しゃぶごまポン酢
追加保存：ボンゴレロッソあさりのトマトソース
追加保存：大盛トマトとニンニクのパスタ
追加保存：高菜と大盛スパゲティ
追加保存：ＺＥＮＢヌードル使用彩り野菜のトマトソース
追加保存：スパゲットーニにんにくトマト
追加保存：小松菜のバター醤油パスタ
追加保存：冷製パスタ生ハムとバジルクリーム
追加保存：冷製パスタトマトと生ハムとチーズ
追加保存：コクと旨味が広がるボンゴレスープパスタ
追加保存：肉の旨味あふれるミートソース
追加保存：大盛明太マヨのスパゲティ
追加保存：肉の旨味あふれるミートソース
追加保存：高崎パスタベスビオ魚介の辛口トマトスープパスタ
追加保存：ソースたっぷり大盛カルボナーラ
追加保存：タルタル盛り鶏唐揚げペペロンチーノ
追加保存：明太クリームパスタフェットチーネ使用
追加保存：ト

In [9]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import os

# CSVファイル名
filename = "seven_eleven_product.csv"

# 栄養成分を正規表現で分解する関数
def parse_nutrition(text):
    nutrition = {
        "calorie": "N/A",
        "protein": "N/A",
        "fat": "N/A",
        "carbohydrate": "N/A",
        "sugar": "N/A",
        "fiber": "N/A",
        "salt": "N/A",
    }

    # 各項目を正規表現で抽出
    patterns = {
        "calorie": r"熱量[:：]\s*([\d\.]+kcal)",
        "protein": r"たんぱく質[:：]\s*([\d\.]+g)",
        "fat": r"脂質[:：]\s*([\d\.]+g)",
        "carbohydrate": r"炭水化物[:：]\s*([\d\.]+g)",
        "sugar": r"糖質[:：]\s*([\d\.]+g)",
        "fiber": r"食物繊維[:：]\s*([\d\.]+g)",
        "salt": r"食塩相当量[:：]\s*([\d\.]+g)",
    }

    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            nutrition[key] = match.group(1)

    return nutrition


# 商品ページから詳細を取得
def scrape_seven_eleven_product(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 商品名
    product_name_div = soup.find('div', class_='item_ttl')
    product_name_tag = product_name_div.find('h1') if product_name_div else None
    product_name = product_name_tag.get_text(strip=True).replace("\u3000", "") if product_name_tag else "N/A"

    # 価格
    price_tag = soup.find('div', class_='item_price')
    price = "N/A"
    if price_tag:
        price_text = price_tag.text.strip()
        price_match = re.search(r'(\d+円（税込\d+円）)', price_text)
        price = price_match.group(1) if price_match else price_text

    # 画像URL
    picture_url = ""
    product_wrap_div = soup.find('div', class_='productWrap')
    if product_wrap_div:
        img_tag = product_wrap_div.find('img')
        if img_tag and img_tag.has_attr("src"):
            picture_url = img_tag['src']

    # 栄養成分
    nutrition_td_text = ""
    for th in soup.find_all("th"):
        if "栄養成分" in th.get_text(strip=True):
            td = th.find_next("td")
            if td:
                nutrition_td_text = td.get_text(" ", strip=True)
                break
    nutrients = parse_nutrition(nutrition_td_text)

    return {
        "product_name": product_name,
        "price": price,
        "picture": picture_url,
        "url": url,
        **nutrients
    }


# CSVに保存（重複回避して追記）
def save_to_csv(data, filename=filename):
    file_exists = os.path.isfile(filename)

    fieldnames = ["product_name", "price", "picture", "url",
                  "calorie", "protein", "fat", "carbohydrate", "sugar", "fiber", "salt"]

    with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # 新規作成時のみヘッダーを書き込む
        if not file_exists:
            writer.writeheader()

        # 既存ファイルの重複チェック
        if file_exists:
            with open(filename, 'r', encoding='utf-8') as f:
                existing = f.read()
            if data["url"] in existing:
                print(f"スキップ（既存）：{data['product_name']}")
                return

        writer.writerow(data)
        print(f"追加保存：{data['product_name']}")


from urllib.parse import urljoin

def scrape_category_all_pages(base_url):
    all_links = set()
    visited_pages = set()
    next_pages = [base_url]

    while next_pages:
        url = next_pages.pop(0)
        if url in visited_pages:
            continue
        visited_pages.add(url)

        print(f"取得中: {url}")
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        # 商品リンク収集
        for a in soup.select("div.list_inner a"):
            href = a.get("href")
            if href and "/products/" in href:
                full_url = urljoin("https://www.sej.co.jp", href)
                all_links.add(full_url)

        # ページャーから次ページ候補を収集
        pager = soup.select("div.pager a")
        for a in pager:
            href = a.get("href")
            if href:
                next_url = urljoin("https://www.sej.co.jp", href)
                if next_url not in visited_pages:
                    next_pages.append(next_url)

    return list(all_links)



# 使用例
if __name__ == "__main__":
    category_url = "https://www.sej.co.jp/products/a/gratin/"
    product_urls = scrape_category_all_pages(category_url)

    print(f"商品数: {len(product_urls)}")
    for url in product_urls:
        try:
            product_info = scrape_seven_eleven_product(url)
            save_to_csv(product_info)
        except Exception as e:
            print(f"エラー：{url} -> {e}")


取得中: https://www.sej.co.jp/products/a/gratin/
取得中: https://www.sej.co.jp/products/a/gratin/2/l15/
取得中: https://www.sej.co.jp/products/a/gratin/1/l15/
商品数: 17
追加保存：チーズ香る濃厚ソースの海老ドリア
追加保存：４種チーズのマカロニグラタン
追加保存：３種チーズのミートソースドリア
追加保存：とろーりチーズのクリームソースドリア
追加保存：３種チーズのミートソースドリア
追加保存：明太クリームドリア
追加保存：かぼちゃのクリームグラタン
追加保存：濃厚ホワイトソースの海老グラタン
追加保存：３種チーズのマカロニグラタン
追加保存：４種きのことベーコンのグラタン
追加保存：濃厚なめらかソースの海老グラタン
追加保存：スパイス香る野菜とチキンの焼きカレードリア
追加保存：トマトソースのオムライスドリア
追加保存：チーズ香る濃厚ソースの海老ドリア
追加保存：とろーり濃厚チーズの海老グラタン
追加保存：３種チーズのミートソースドリア
追加保存：濃厚なめらかソースの海老グラタン


In [10]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import os

# CSVファイル名
filename = "seven_eleven_product.csv"

# 栄養成分を正規表現で分解する関数
def parse_nutrition(text):
    nutrition = {
        "calorie": "N/A",
        "protein": "N/A",
        "fat": "N/A",
        "carbohydrate": "N/A",
        "sugar": "N/A",
        "fiber": "N/A",
        "salt": "N/A",
    }

    # 各項目を正規表現で抽出
    patterns = {
        "calorie": r"熱量[:：]\s*([\d\.]+kcal)",
        "protein": r"たんぱく質[:：]\s*([\d\.]+g)",
        "fat": r"脂質[:：]\s*([\d\.]+g)",
        "carbohydrate": r"炭水化物[:：]\s*([\d\.]+g)",
        "sugar": r"糖質[:：]\s*([\d\.]+g)",
        "fiber": r"食物繊維[:：]\s*([\d\.]+g)",
        "salt": r"食塩相当量[:：]\s*([\d\.]+g)",
    }

    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            nutrition[key] = match.group(1)

    return nutrition


# 商品ページから詳細を取得
def scrape_seven_eleven_product(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 商品名
    product_name_div = soup.find('div', class_='item_ttl')
    product_name_tag = product_name_div.find('h1') if product_name_div else None
    product_name = product_name_tag.get_text(strip=True).replace("\u3000", "") if product_name_tag else "N/A"

    # 価格
    price_tag = soup.find('div', class_='item_price')
    price = "N/A"
    if price_tag:
        price_text = price_tag.text.strip()
        price_match = re.search(r'(\d+円（税込\d+円）)', price_text)
        price = price_match.group(1) if price_match else price_text

    # 画像URL
    picture_url = ""
    product_wrap_div = soup.find('div', class_='productWrap')
    if product_wrap_div:
        img_tag = product_wrap_div.find('img')
        if img_tag and img_tag.has_attr("src"):
            picture_url = img_tag['src']

    # 栄養成分
    nutrition_td_text = ""
    for th in soup.find_all("th"):
        if "栄養成分" in th.get_text(strip=True):
            td = th.find_next("td")
            if td:
                nutrition_td_text = td.get_text(" ", strip=True)
                break
    nutrients = parse_nutrition(nutrition_td_text)

    return {
        "product_name": product_name,
        "price": price,
        "picture": picture_url,
        "url": url,
        **nutrients
    }


# CSVに保存（重複回避して追記）
def save_to_csv(data, filename=filename):
    file_exists = os.path.isfile(filename)

    fieldnames = ["product_name", "price", "picture", "url",
                  "calorie", "protein", "fat", "carbohydrate", "sugar", "fiber", "salt"]

    with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # 新規作成時のみヘッダーを書き込む
        if not file_exists:
            writer.writeheader()

        # 既存ファイルの重複チェック
        if file_exists:
            with open(filename, 'r', encoding='utf-8') as f:
                existing = f.read()
            if data["url"] in existing:
                print(f"スキップ（既存）：{data['product_name']}")
                return

        writer.writerow(data)
        print(f"追加保存：{data['product_name']}")


from urllib.parse import urljoin

def scrape_category_all_pages(base_url):
    all_links = set()
    visited_pages = set()
    next_pages = [base_url]

    while next_pages:
        url = next_pages.pop(0)
        if url in visited_pages:
            continue
        visited_pages.add(url)

        print(f"取得中: {url}")
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        # 商品リンク収集
        for a in soup.select("div.list_inner a"):
            href = a.get("href")
            if href and "/products/" in href:
                full_url = urljoin("https://www.sej.co.jp", href)
                all_links.add(full_url)

        # ページャーから次ページ候補を収集
        pager = soup.select("div.pager a")
        for a in pager:
            href = a.get("href")
            if href:
                next_url = urljoin("https://www.sej.co.jp", href)
                if next_url not in visited_pages:
                    next_pages.append(next_url)

    return list(all_links)



# 使用例
if __name__ == "__main__":
    category_url = "https://www.sej.co.jp/products/a/dailydish/"
    product_urls = scrape_category_all_pages(category_url)

    print(f"商品数: {len(product_urls)}")
    for url in product_urls:
        try:
            product_info = scrape_seven_eleven_product(url)
            save_to_csv(product_info)
        except Exception as e:
            print(f"エラー：{url} -> {e}")


取得中: https://www.sej.co.jp/products/a/dailydish/
取得中: https://www.sej.co.jp/products/a/dailydish/2/l15/
取得中: https://www.sej.co.jp/products/a/dailydish/3/l15/
取得中: https://www.sej.co.jp/products/a/dailydish/4/l15/
取得中: https://www.sej.co.jp/products/a/dailydish/5/l15/
取得中: https://www.sej.co.jp/products/a/dailydish/6/l15/
取得中: https://www.sej.co.jp/products/a/dailydish/7/l15/
取得中: https://www.sej.co.jp/products/a/dailydish/8/l15/
取得中: https://www.sej.co.jp/products/a/dailydish/9/l15/
取得中: https://www.sej.co.jp/products/a/dailydish/10/l15/
取得中: https://www.sej.co.jp/products/a/dailydish/1/l15/
取得中: https://www.sej.co.jp/products/a/dailydish/11/l15/
取得中: https://www.sej.co.jp/products/a/dailydish/12/l15/
取得中: https://www.sej.co.jp/products/a/dailydish/13/l15/
取得中: https://www.sej.co.jp/products/a/dailydish/14/l15/
取得中: https://www.sej.co.jp/products/a/dailydish/15/l15/
商品数: 218
追加保存：旨辛やみつき豚キムチ
追加保存：青森県産地養豚のコクうま豚汁
追加保存：７プレミアムおでん（カップ）
追加保存：四川風麻婆豆腐
追加保存：合わせ味噌仕立ての具だくさん豚汁
追加保存：炭火焼鳥しお（七味付き）
追加

In [11]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import os

# CSVファイル名
filename = "seven_eleven_product.csv"

# 栄養成分を正規表現で分解する関数
def parse_nutrition(text):
    nutrition = {
        "calorie": "N/A",
        "protein": "N/A",
        "fat": "N/A",
        "carbohydrate": "N/A",
        "sugar": "N/A",
        "fiber": "N/A",
        "salt": "N/A",
    }

    # 各項目を正規表現で抽出
    patterns = {
        "calorie": r"熱量[:：]\s*([\d\.]+kcal)",
        "protein": r"たんぱく質[:：]\s*([\d\.]+g)",
        "fat": r"脂質[:：]\s*([\d\.]+g)",
        "carbohydrate": r"炭水化物[:：]\s*([\d\.]+g)",
        "sugar": r"糖質[:：]\s*([\d\.]+g)",
        "fiber": r"食物繊維[:：]\s*([\d\.]+g)",
        "salt": r"食塩相当量[:：]\s*([\d\.]+g)",
    }

    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            nutrition[key] = match.group(1)

    return nutrition


# 商品ページから詳細を取得
def scrape_seven_eleven_product(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 商品名
    product_name_div = soup.find('div', class_='item_ttl')
    product_name_tag = product_name_div.find('h1') if product_name_div else None
    product_name = product_name_tag.get_text(strip=True).replace("\u3000", "") if product_name_tag else "N/A"

    # 価格
    price_tag = soup.find('div', class_='item_price')
    price = "N/A"
    if price_tag:
        price_text = price_tag.text.strip()
        price_match = re.search(r'(\d+円（税込\d+円）)', price_text)
        price = price_match.group(1) if price_match else price_text

    # 画像URL
    picture_url = ""
    product_wrap_div = soup.find('div', class_='productWrap')
    if product_wrap_div:
        img_tag = product_wrap_div.find('img')
        if img_tag and img_tag.has_attr("src"):
            picture_url = img_tag['src']

    # 栄養成分
    nutrition_td_text = ""
    for th in soup.find_all("th"):
        if "栄養成分" in th.get_text(strip=True):
            td = th.find_next("td")
            if td:
                nutrition_td_text = td.get_text(" ", strip=True)
                break
    nutrients = parse_nutrition(nutrition_td_text)

    return {
        "product_name": product_name,
        "price": price,
        "picture": picture_url,
        "url": url,
        **nutrients
    }


# CSVに保存（重複回避して追記）
def save_to_csv(data, filename=filename):
    file_exists = os.path.isfile(filename)

    fieldnames = ["product_name", "price", "picture", "url",
                  "calorie", "protein", "fat", "carbohydrate", "sugar", "fiber", "salt"]

    with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # 新規作成時のみヘッダーを書き込む
        if not file_exists:
            writer.writeheader()

        # 既存ファイルの重複チェック
        if file_exists:
            with open(filename, 'r', encoding='utf-8') as f:
                existing = f.read()
            if data["url"] in existing:
                print(f"スキップ（既存）：{data['product_name']}")
                return

        writer.writerow(data)
        print(f"追加保存：{data['product_name']}")


from urllib.parse import urljoin

def scrape_category_all_pages(base_url):
    all_links = set()
    visited_pages = set()
    next_pages = [base_url]

    while next_pages:
        url = next_pages.pop(0)
        if url in visited_pages:
            continue
        visited_pages.add(url)

        print(f"取得中: {url}")
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        # 商品リンク収集
        for a in soup.select("div.list_inner a"):
            href = a.get("href")
            if href and "/products/" in href:
                full_url = urljoin("https://www.sej.co.jp", href)
                all_links.add(full_url)

        # ページャーから次ページ候補を収集
        pager = soup.select("div.pager a")
        for a in pager:
            href = a.get("href")
            if href:
                next_url = urljoin("https://www.sej.co.jp", href)
                if next_url not in visited_pages:
                    next_pages.append(next_url)

    return list(all_links)



# 使用例
if __name__ == "__main__":
    category_url = "https://www.sej.co.jp/products/a/salad/"
    product_urls = scrape_category_all_pages(category_url)

    print(f"商品数: {len(product_urls)}")
    for url in product_urls:
        try:
            product_info = scrape_seven_eleven_product(url)
            save_to_csv(product_info)
        except Exception as e:
            print(f"エラー：{url} -> {e}")


取得中: https://www.sej.co.jp/products/a/salad/
商品数: 24
追加保存：７プレミアムコールスロー
追加保存：プリプリ海老のパスタサラダ
追加保存：たんぱく質が摂れる鶏むね肉サラダ
追加保存：７プレミアムマカロニサラダ
追加保存：たんぱく質が摂れる鶏むね肉サラダ
追加保存：７プレミアムごぼうサラダ
追加保存：アスパラベーコン＆タルタルポテト
追加保存：７プレミアムごぼうサラダ
追加保存：７プレミアムコールスロー
追加保存：７プレミアムたまごサラダ
追加保存：いわしのマリネオリーブオイル仕立て
追加保存：７プレミアム千切りキャベツ
追加保存：比叡ゆばとブロッコリーのおかか和え
追加保存：大葉と海老の明太子クリームパスタサラダ
追加保存：７プレミアムレタスサラダ
追加保存：７プレミアムマカロニサラダ
追加保存：７プレミアム大根サラダ
追加保存：７プレミアム千切りキャベツ
追加保存：信州味噌使用肉味噌ラーメンサラダ
追加保存：７プレミアムポテトサラダ
追加保存：大山どりのごまごまチキン
追加保存：但馬の味どりのシャキシャキ野菜サラダ
追加保存：ピリッ！とからっキュウ
追加保存：ポン酢で食べる砂ずりと玉ねぎスライス


In [12]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import os

# CSVファイル名
filename = "seven_eleven_product.csv"

# 栄養成分を正規表現で分解する関数
def parse_nutrition(text):
    nutrition = {
        "calorie": "N/A",
        "protein": "N/A",
        "fat": "N/A",
        "carbohydrate": "N/A",
        "sugar": "N/A",
        "fiber": "N/A",
        "salt": "N/A",
    }

    # 各項目を正規表現で抽出
    patterns = {
        "calorie": r"熱量[:：]\s*([\d\.]+kcal)",
        "protein": r"たんぱく質[:：]\s*([\d\.]+g)",
        "fat": r"脂質[:：]\s*([\d\.]+g)",
        "carbohydrate": r"炭水化物[:：]\s*([\d\.]+g)",
        "sugar": r"糖質[:：]\s*([\d\.]+g)",
        "fiber": r"食物繊維[:：]\s*([\d\.]+g)",
        "salt": r"食塩相当量[:：]\s*([\d\.]+g)",
    }

    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            nutrition[key] = match.group(1)

    return nutrition


# 商品ページから詳細を取得
def scrape_seven_eleven_product(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 商品名
    product_name_div = soup.find('div', class_='item_ttl')
    product_name_tag = product_name_div.find('h1') if product_name_div else None
    product_name = product_name_tag.get_text(strip=True).replace("\u3000", "") if product_name_tag else "N/A"

    # 価格
    price_tag = soup.find('div', class_='item_price')
    price = "N/A"
    if price_tag:
        price_text = price_tag.text.strip()
        price_match = re.search(r'(\d+円（税込\d+円）)', price_text)
        price = price_match.group(1) if price_match else price_text

    # 画像URL
    picture_url = ""
    product_wrap_div = soup.find('div', class_='productWrap')
    if product_wrap_div:
        img_tag = product_wrap_div.find('img')
        if img_tag and img_tag.has_attr("src"):
            picture_url = img_tag['src']

    # 栄養成分
    nutrition_td_text = ""
    for th in soup.find_all("th"):
        if "栄養成分" in th.get_text(strip=True):
            td = th.find_next("td")
            if td:
                nutrition_td_text = td.get_text(" ", strip=True)
                break
    nutrients = parse_nutrition(nutrition_td_text)

    return {
        "product_name": product_name,
        "price": price,
        "picture": picture_url,
        "url": url,
        **nutrients
    }


# CSVに保存（重複回避して追記）
def save_to_csv(data, filename=filename):
    file_exists = os.path.isfile(filename)

    fieldnames = ["product_name", "price", "picture", "url",
                  "calorie", "protein", "fat", "carbohydrate", "sugar", "fiber", "salt"]

    with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # 新規作成時のみヘッダーを書き込む
        if not file_exists:
            writer.writeheader()

        # 既存ファイルの重複チェック
        if file_exists:
            with open(filename, 'r', encoding='utf-8') as f:
                existing = f.read()
            if data["url"] in existing:
                print(f"スキップ（既存）：{data['product_name']}")
                return

        writer.writerow(data)
        print(f"追加保存：{data['product_name']}")


from urllib.parse import urljoin

def scrape_category_all_pages(base_url):
    all_links = set()
    visited_pages = set()
    next_pages = [base_url]

    while next_pages:
        url = next_pages.pop(0)
        if url in visited_pages:
            continue
        visited_pages.add(url)

        print(f"取得中: {url}")
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        # 商品リンク収集
        for a in soup.select("div.list_inner a"):
            href = a.get("href")
            if href and "/products/" in href:
                full_url = urljoin("https://www.sej.co.jp", href)
                all_links.add(full_url)

        # ページャーから次ページ候補を収集
        pager = soup.select("div.pager a")
        for a in pager:
            href = a.get("href")
            if href:
                next_url = urljoin("https://www.sej.co.jp", href)
                if next_url not in visited_pages:
                    next_pages.append(next_url)

    return list(all_links)



# 使用例
if __name__ == "__main__":
    category_url = "https://www.sej.co.jp/products/a/sweets/"
    product_urls = scrape_category_all_pages(category_url)

    print(f"商品数: {len(product_urls)}")
    for url in product_urls:
        try:
            product_info = scrape_seven_eleven_product(url)
            save_to_csv(product_info)
        except Exception as e:
            print(f"エラー：{url} -> {e}")


取得中: https://www.sej.co.jp/products/a/sweets/
商品数: 12
追加保存：ひとくち包みみたらし
追加保存：近江ほうじ茶のチーズケーキ
追加保存：阿蘇小国ジャージー牛乳使用ミルクムースシュー
追加保存：ずっしり塩大福
追加保存：出雲ぜんざい学会監修ひんやり出雲ぜんざい
追加保存：バター香るフィナンシェ
追加保存：ずっしり草もち
追加保存：白バラ牛乳使用ミルクプリンケ―キ
追加保存：ずっしり豆大福
追加保存：たっぷりホイップのダブルシュー近江ほうじ茶
追加保存：ずんだが主役のお団子
追加保存：くちどけクリーム＆シフォンケーキ


In [13]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import os

# CSVファイル名
filename = "seven_eleven_product.csv"

# 栄養成分を正規表現で分解する関数
def parse_nutrition(text):
    nutrition = {
        "calorie": "N/A",
        "protein": "N/A",
        "fat": "N/A",
        "carbohydrate": "N/A",
        "sugar": "N/A",
        "fiber": "N/A",
        "salt": "N/A",
    }

    # 各項目を正規表現で抽出
    patterns = {
        "calorie": r"熱量[:：]\s*([\d\.]+kcal)",
        "protein": r"たんぱく質[:：]\s*([\d\.]+g)",
        "fat": r"脂質[:：]\s*([\d\.]+g)",
        "carbohydrate": r"炭水化物[:：]\s*([\d\.]+g)",
        "sugar": r"糖質[:：]\s*([\d\.]+g)",
        "fiber": r"食物繊維[:：]\s*([\d\.]+g)",
        "salt": r"食塩相当量[:：]\s*([\d\.]+g)",
    }

    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            nutrition[key] = match.group(1)

    return nutrition


# 商品ページから詳細を取得
def scrape_seven_eleven_product(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 商品名
    product_name_div = soup.find('div', class_='item_ttl')
    product_name_tag = product_name_div.find('h1') if product_name_div else None
    product_name = product_name_tag.get_text(strip=True).replace("\u3000", "") if product_name_tag else "N/A"

    # 価格
    price_tag = soup.find('div', class_='item_price')
    price = "N/A"
    if price_tag:
        price_text = price_tag.text.strip()
        price_match = re.search(r'(\d+円（税込\d+円）)', price_text)
        price = price_match.group(1) if price_match else price_text

    # 画像URL
    picture_url = ""
    product_wrap_div = soup.find('div', class_='productWrap')
    if product_wrap_div:
        img_tag = product_wrap_div.find('img')
        if img_tag and img_tag.has_attr("src"):
            picture_url = img_tag['src']

    # 栄養成分
    nutrition_td_text = ""
    for th in soup.find_all("th"):
        if "栄養成分" in th.get_text(strip=True):
            td = th.find_next("td")
            if td:
                nutrition_td_text = td.get_text(" ", strip=True)
                break
    nutrients = parse_nutrition(nutrition_td_text)

    return {
        "product_name": product_name,
        "price": price,
        "picture": picture_url,
        "url": url,
        **nutrients
    }


# CSVに保存（重複回避して追記）
def save_to_csv(data, filename=filename):
    file_exists = os.path.isfile(filename)

    fieldnames = ["product_name", "price", "picture", "url",
                  "calorie", "protein", "fat", "carbohydrate", "sugar", "fiber", "salt"]

    with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # 新規作成時のみヘッダーを書き込む
        if not file_exists:
            writer.writeheader()

        # 既存ファイルの重複チェック
        if file_exists:
            with open(filename, 'r', encoding='utf-8') as f:
                existing = f.read()
            if data["url"] in existing:
                print(f"スキップ（既存）：{data['product_name']}")
                return

        writer.writerow(data)
        print(f"追加保存：{data['product_name']}")


from urllib.parse import urljoin

def scrape_category_all_pages(base_url):
    all_links = set()
    visited_pages = set()
    next_pages = [base_url]

    while next_pages:
        url = next_pages.pop(0)
        if url in visited_pages:
            continue
        visited_pages.add(url)

        print(f"取得中: {url}")
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        # 商品リンク収集
        for a in soup.select("div.list_inner a"):
            href = a.get("href")
            if href and "/products/" in href:
                full_url = urljoin("https://www.sej.co.jp", href)
                all_links.add(full_url)

        # ページャーから次ページ候補を収集
        pager = soup.select("div.pager a")
        for a in pager:
            href = a.get("href")
            if href:
                next_url = urljoin("https://www.sej.co.jp", href)
                if next_url not in visited_pages:
                    next_pages.append(next_url)

    return list(all_links)



# 使用例
if __name__ == "__main__":
    category_url = "https://www.sej.co.jp/products/a/hotsnack/"
    product_urls = scrape_category_all_pages(category_url)

    print(f"商品数: {len(product_urls)}")
    for url in product_urls:
        try:
            product_info = scrape_seven_eleven_product(url)
            save_to_csv(product_info)
        except Exception as e:
            print(f"エラー：{url} -> {e}")


取得中: https://www.sej.co.jp/products/a/hotsnack/
商品数: 12
追加保存：【盛盛】若鶏のからあげ（むね）２０個
追加保存：ＢＩＧポークフランク
追加保存：炭火焼き鳥（塩）
追加保存：若鶏のからあげ（もも）５個入り
追加保存：ジューシー粗挽きソーセージ
追加保存：【盛盛】若鶏のからあげ（もも）２０個
追加保存：スパイスチキンレッド
追加保存：若鶏のからあげ（むね）５個入り
追加保存：ザクチキ（背徳のマシマシ）
追加保存：北海道産じゃがいもの牛肉コロッケ
追加保存：スパイスチキン
追加保存：炭火焼き鳥（タレ）


In [14]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import os

# CSVファイル名
filename = "seven_eleven_product.csv"

# 栄養成分を正規表現で分解する関数
def parse_nutrition(text):
    nutrition = {
        "calorie": "N/A",
        "protein": "N/A",
        "fat": "N/A",
        "carbohydrate": "N/A",
        "sugar": "N/A",
        "fiber": "N/A",
        "salt": "N/A",
    }

    # 各項目を正規表現で抽出
    patterns = {
        "calorie": r"熱量[:：]\s*([\d\.]+kcal)",
        "protein": r"たんぱく質[:：]\s*([\d\.]+g)",
        "fat": r"脂質[:：]\s*([\d\.]+g)",
        "carbohydrate": r"炭水化物[:：]\s*([\d\.]+g)",
        "sugar": r"糖質[:：]\s*([\d\.]+g)",
        "fiber": r"食物繊維[:：]\s*([\d\.]+g)",
        "salt": r"食塩相当量[:：]\s*([\d\.]+g)",
    }

    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            nutrition[key] = match.group(1)

    return nutrition


# 商品ページから詳細を取得
def scrape_seven_eleven_product(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 商品名
    product_name_div = soup.find('div', class_='item_ttl')
    product_name_tag = product_name_div.find('h1') if product_name_div else None
    product_name = product_name_tag.get_text(strip=True).replace("\u3000", "") if product_name_tag else "N/A"

    # 価格
    price_tag = soup.find('div', class_='item_price')
    price = "N/A"
    if price_tag:
        price_text = price_tag.text.strip()
        price_match = re.search(r'(\d+円（税込\d+円）)', price_text)
        price = price_match.group(1) if price_match else price_text

    # 画像URL
    picture_url = ""
    product_wrap_div = soup.find('div', class_='productWrap')
    if product_wrap_div:
        img_tag = product_wrap_div.find('img')
        if img_tag and img_tag.has_attr("src"):
            picture_url = img_tag['src']

    # 栄養成分
    nutrition_td_text = ""
    for th in soup.find_all("th"):
        if "栄養成分" in th.get_text(strip=True):
            td = th.find_next("td")
            if td:
                nutrition_td_text = td.get_text(" ", strip=True)
                break
    nutrients = parse_nutrition(nutrition_td_text)

    return {
        "product_name": product_name,
        "price": price,
        "picture": picture_url,
        "url": url,
        **nutrients
    }


# CSVに保存（重複回避して追記）
def save_to_csv(data, filename=filename):
    file_exists = os.path.isfile(filename)

    fieldnames = ["product_name", "price", "picture", "url",
                  "calorie", "protein", "fat", "carbohydrate", "sugar", "fiber", "salt"]

    with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # 新規作成時のみヘッダーを書き込む
        if not file_exists:
            writer.writeheader()

        # 既存ファイルの重複チェック
        if file_exists:
            with open(filename, 'r', encoding='utf-8') as f:
                existing = f.read()
            if data["url"] in existing:
                print(f"スキップ（既存）：{data['product_name']}")
                return

        writer.writerow(data)
        print(f"追加保存：{data['product_name']}")


from urllib.parse import urljoin

def scrape_category_all_pages(base_url):
    all_links = set()
    visited_pages = set()
    next_pages = [base_url]

    while next_pages:
        url = next_pages.pop(0)
        if url in visited_pages:
            continue
        visited_pages.add(url)

        print(f"取得中: {url}")
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        # 商品リンク収集
        for a in soup.select("div.list_inner a"):
            href = a.get("href")
            if href and "/products/" in href:
                full_url = urljoin("https://www.sej.co.jp", href)
                all_links.add(full_url)

        # ページャーから次ページ候補を収集
        pager = soup.select("div.pager a")
        for a in pager:
            href = a.get("href")
            if href:
                next_url = urljoin("https://www.sej.co.jp", href)
                if next_url not in visited_pages:
                    next_pages.append(next_url)

    return list(all_links)



# 使用例
if __name__ == "__main__":
    category_url = "https://www.sej.co.jp/products/a/oden/"
    product_urls = scrape_category_all_pages(category_url)

    print(f"商品数: {len(product_urls)}")
    for url in product_urls:
        try:
            product_info = scrape_seven_eleven_product(url)
            save_to_csv(product_info)
        except Exception as e:
            print(f"エラー：{url} -> {e}")


取得中: https://www.sej.co.jp/products/a/oden/
取得中: https://www.sej.co.jp/products/a/oden/2/l15/
取得中: https://www.sej.co.jp/products/a/oden/1/l15/
商品数: 16
追加保存：おでんもっちりちくわぶ
追加保存：おでん味しみ大根
追加保存：おでん味しみこんにゃく
追加保存：おでん焼豆腐
追加保存：おでん味しみ白こんにゃく
追加保存：おでん味しみたまご
追加保存：おでん木綿厚揚げ
追加保存：おでんなんこつ入り鶏つくね串
追加保存：おでん味しみ白滝
追加保存：おでんウインナー巻
追加保存：おでん牛すじ串
追加保存：おでんふんわりはんぺん
追加保存：おでんお餅の巾着
追加保存：おでん味しみ糸こん
追加保存：おでん５種野菜のさつま揚げ
追加保存：おでん味しみ大根


In [15]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import os

# CSVファイル名
filename = "seven_eleven_product.csv"

# 栄養成分を正規表現で分解する関数
def parse_nutrition(text):
    nutrition = {
        "calorie": "N/A",
        "protein": "N/A",
        "fat": "N/A",
        "carbohydrate": "N/A",
        "sugar": "N/A",
        "fiber": "N/A",
        "salt": "N/A",
    }

    # 各項目を正規表現で抽出
    patterns = {
        "calorie": r"熱量[:：]\s*([\d\.]+kcal)",
        "protein": r"たんぱく質[:：]\s*([\d\.]+g)",
        "fat": r"脂質[:：]\s*([\d\.]+g)",
        "carbohydrate": r"炭水化物[:：]\s*([\d\.]+g)",
        "sugar": r"糖質[:：]\s*([\d\.]+g)",
        "fiber": r"食物繊維[:：]\s*([\d\.]+g)",
        "salt": r"食塩相当量[:：]\s*([\d\.]+g)",
    }

    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            nutrition[key] = match.group(1)

    return nutrition


# 商品ページから詳細を取得
def scrape_seven_eleven_product(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 商品名
    product_name_div = soup.find('div', class_='item_ttl')
    product_name_tag = product_name_div.find('h1') if product_name_div else None
    product_name = product_name_tag.get_text(strip=True).replace("\u3000", "") if product_name_tag else "N/A"

    # 価格
    price_tag = soup.find('div', class_='item_price')
    price = "N/A"
    if price_tag:
        price_text = price_tag.text.strip()
        price_match = re.search(r'(\d+円（税込\d+円）)', price_text)
        price = price_match.group(1) if price_match else price_text

    # 画像URL
    picture_url = ""
    product_wrap_div = soup.find('div', class_='productWrap')
    if product_wrap_div:
        img_tag = product_wrap_div.find('img')
        if img_tag and img_tag.has_attr("src"):
            picture_url = img_tag['src']

    # 栄養成分
    nutrition_td_text = ""
    for th in soup.find_all("th"):
        if "栄養成分" in th.get_text(strip=True):
            td = th.find_next("td")
            if td:
                nutrition_td_text = td.get_text(" ", strip=True)
                break
    nutrients = parse_nutrition(nutrition_td_text)

    return {
        "product_name": product_name,
        "price": price,
        "picture": picture_url,
        "url": url,
        **nutrients
    }


# CSVに保存（重複回避して追記）
def save_to_csv(data, filename=filename):
    file_exists = os.path.isfile(filename)

    fieldnames = ["product_name", "price", "picture", "url",
                  "calorie", "protein", "fat", "carbohydrate", "sugar", "fiber", "salt"]

    with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # 新規作成時のみヘッダーを書き込む
        if not file_exists:
            writer.writeheader()

        # 既存ファイルの重複チェック
        if file_exists:
            with open(filename, 'r', encoding='utf-8') as f:
                existing = f.read()
            if data["url"] in existing:
                print(f"スキップ（既存）：{data['product_name']}")
                return

        writer.writerow(data)
        print(f"追加保存：{data['product_name']}")


from urllib.parse import urljoin

def scrape_category_all_pages(base_url):
    all_links = set()
    visited_pages = set()
    next_pages = [base_url]

    while next_pages:
        url = next_pages.pop(0)
        if url in visited_pages:
            continue
        visited_pages.add(url)

        print(f"取得中: {url}")
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        # 商品リンク収集
        for a in soup.select("div.list_inner a"):
            href = a.get("href")
            if href and "/products/" in href:
                full_url = urljoin("https://www.sej.co.jp", href)
                all_links.add(full_url)

        # ページャーから次ページ候補を収集
        pager = soup.select("div.pager a")
        for a in pager:
            href = a.get("href")
            if href:
                next_url = urljoin("https://www.sej.co.jp", href)
                if next_url not in visited_pages:
                    next_pages.append(next_url)

    return list(all_links)



# 使用例
if __name__ == "__main__":
    category_url = "https://www.sej.co.jp/products/a/ice_cream/"
    product_urls = scrape_category_all_pages(category_url)

    print(f"商品数: {len(product_urls)}")
    for url in product_urls:
        try:
            product_info = scrape_seven_eleven_product(url)
            save_to_csv(product_info)
        except Exception as e:
            print(f"エラー：{url} -> {e}")


取得中: https://www.sej.co.jp/products/a/ice_cream/
商品数: 12
追加保存：森永ピノダブルショコラ
追加保存：ハーゲンダッツアソートボックス世界のデザートセレクション
追加保存：ロッテクーリッシュ濃密みかん
追加保存：７プレミアム北海道バニラバーマルチ
追加保存：７プレミアムワッフルコーンチョコ＆ミルク
追加保存：７プレミアムシュガーコーンマルチ
追加保存：フタバ食品ダンディーチョコバニラマルチ
追加保存：７Ｐチョコレートバーりんご
追加保存：ハーゲンダッツアソートボックスラバーズアソート
追加保存：赤城セルフクラッシュクッキークリーム
追加保存：７プレミアム４種のフルーツバーマルチ
追加保存：７プレミアム生チョコアイス


In [16]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import os

# CSVファイル名
filename = "seven_eleven_product.csv"

# 栄養成分を正規表現で分解する関数
def parse_nutrition(text):
    nutrition = {
        "calorie": "N/A",
        "protein": "N/A",
        "fat": "N/A",
        "carbohydrate": "N/A",
        "sugar": "N/A",
        "fiber": "N/A",
        "salt": "N/A",
    }

    # 各項目を正規表現で抽出
    patterns = {
        "calorie": r"熱量[:：]\s*([\d\.]+kcal)",
        "protein": r"たんぱく質[:：]\s*([\d\.]+g)",
        "fat": r"脂質[:：]\s*([\d\.]+g)",
        "carbohydrate": r"炭水化物[:：]\s*([\d\.]+g)",
        "sugar": r"糖質[:：]\s*([\d\.]+g)",
        "fiber": r"食物繊維[:：]\s*([\d\.]+g)",
        "salt": r"食塩相当量[:：]\s*([\d\.]+g)",
    }

    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            nutrition[key] = match.group(1)

    return nutrition


# 商品ページから詳細を取得
def scrape_seven_eleven_product(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 商品名
    product_name_div = soup.find('div', class_='item_ttl')
    product_name_tag = product_name_div.find('h1') if product_name_div else None
    product_name = product_name_tag.get_text(strip=True).replace("\u3000", "") if product_name_tag else "N/A"

    # 価格
    price_tag = soup.find('div', class_='item_price')
    price = "N/A"
    if price_tag:
        price_text = price_tag.text.strip()
        price_match = re.search(r'(\d+円（税込\d+円）)', price_text)
        price = price_match.group(1) if price_match else price_text

    # 画像URL
    picture_url = ""
    product_wrap_div = soup.find('div', class_='productWrap')
    if product_wrap_div:
        img_tag = product_wrap_div.find('img')
        if img_tag and img_tag.has_attr("src"):
            picture_url = img_tag['src']

    # 栄養成分
    nutrition_td_text = ""
    for th in soup.find_all("th"):
        if "栄養成分" in th.get_text(strip=True):
            td = th.find_next("td")
            if td:
                nutrition_td_text = td.get_text(" ", strip=True)
                break
    nutrients = parse_nutrition(nutrition_td_text)

    return {
        "product_name": product_name,
        "price": price,
        "picture": picture_url,
        "url": url,
        **nutrients
    }


# CSVに保存（重複回避して追記）
def save_to_csv(data, filename=filename):
    file_exists = os.path.isfile(filename)

    fieldnames = ["product_name", "price", "picture", "url",
                  "calorie", "protein", "fat", "carbohydrate", "sugar", "fiber", "salt"]

    with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # 新規作成時のみヘッダーを書き込む
        if not file_exists:
            writer.writeheader()

        # 既存ファイルの重複チェック
        if file_exists:
            with open(filename, 'r', encoding='utf-8') as f:
                existing = f.read()
            if data["url"] in existing:
                print(f"スキップ（既存）：{data['product_name']}")
                return

        writer.writerow(data)
        print(f"追加保存：{data['product_name']}")


from urllib.parse import urljoin

def scrape_category_all_pages(base_url):
    all_links = set()
    visited_pages = set()
    next_pages = [base_url]

    while next_pages:
        url = next_pages.pop(0)
        if url in visited_pages:
            continue
        visited_pages.add(url)

        print(f"取得中: {url}")
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        # 商品リンク収集
        for a in soup.select("div.list_inner a"):
            href = a.get("href")
            if href and "/products/" in href:
                full_url = urljoin("https://www.sej.co.jp", href)
                all_links.add(full_url)

        # ページャーから次ページ候補を収集
        pager = soup.select("div.pager a")
        for a in pager:
            href = a.get("href")
            if href:
                next_url = urljoin("https://www.sej.co.jp", href)
                if next_url not in visited_pages:
                    next_pages.append(next_url)

    return list(all_links)



# 使用例
if __name__ == "__main__":
    category_url = "https://www.sej.co.jp/products/a/chukaman/"
    product_urls = scrape_category_all_pages(category_url)

    print(f"商品数: {len(product_urls)}")
    for url in product_urls:
        try:
            product_info = scrape_seven_eleven_product(url)
            save_to_csv(product_info)
        except Exception as e:
            print(f"エラー：{url} -> {e}")


取得中: https://www.sej.co.jp/products/a/chukaman/
商品数: 7
追加保存：ふんわり×ごろっと肉まん
追加保存：もちふわ×とろ～りピザまん
追加保存：ふわふわ×とろっと濃厚ごまあんまん
追加保存：もちもち×ずっしり大入り豚まん
追加保存：もっちり×ジューシー特製豚まん
追加保存：じゃがまるくん（ポテト＆ミート）
追加保存：もちふわ×つぶつぶつぶあんまん


In [22]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import os

# CSVファイル名
filename = "seven_eleven_product.csv"

# 栄養成分を正規表現で分解する関数
def parse_nutrition(text):
    nutrition = {
        "calorie": "N/A",
        "protein": "N/A",
        "fat": "N/A",
        "carbohydrate": "N/A",
        "sugar": "N/A",
        "fiber": "N/A",
        "salt": "N/A",
    }

    # 各項目を正規表現で抽出
    patterns = {
        "calorie": r"熱量[:：]\s*([\d\.]+kcal)",
        "protein": r"たんぱく質[:：]\s*([\d\.]+g)",
        "fat": r"脂質[:：]\s*([\d\.]+g)",
        "carbohydrate": r"炭水化物[:：]\s*([\d\.]+g)",
        "sugar": r"糖質[:：]\s*([\d\.]+g)",
        "fiber": r"食物繊維[:：]\s*([\d\.]+g)",
        "salt": r"食塩相当量[:：]\s*([\d\.]+g)",
    }

    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            nutrition[key] = match.group(1)

    return nutrition


# 商品ページから詳細を取得
def scrape_seven_eleven_product(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 商品名
    product_name_div = soup.find('div', class_='item_ttl')
    product_name_tag = product_name_div.find('h1') if product_name_div else None
    product_name = product_name_tag.get_text(strip=True).replace("\u3000", "") if product_name_tag else "N/A"

    # 価格
    price_tag = soup.find('div', class_='item_price')
    price = "N/A"
    if price_tag:
        price_text = price_tag.text.strip()
        price_match = re.search(r'(\d+円（税込\d+円）)', price_text)
        price = price_match.group(1) if price_match else price_text

    # 画像URL
    picture_url = ""
    product_wrap_div = soup.find('div', class_='productWrap')
    if product_wrap_div:
        img_tag = product_wrap_div.find('img')
        if img_tag and img_tag.has_attr("src"):
            picture_url = img_tag['src']

    # 栄養成分
    nutrition_td_text = ""
    for th in soup.find_all("th"):
        if "栄養成分" in th.get_text(strip=True):
            td = th.find_next("td")
            if td:
                nutrition_td_text = td.get_text(" ", strip=True)
                break
    nutrients = parse_nutrition(nutrition_td_text)

    return {
        "product_name": product_name,
        "price": price,
        "picture": picture_url,
        "url": url,
        **nutrients
    }


# CSVに保存（重複回避して追記）
def save_to_csv(data, filename=filename):
    file_exists = os.path.isfile(filename)

    fieldnames = ["product_name", "price", "picture", "url",
                  "calorie", "protein", "fat", "carbohydrate", "sugar", "fiber", "salt"]

    with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # 新規作成時のみヘッダーを書き込む
        if not file_exists:
            writer.writeheader()

        # 既存ファイルの重複チェック
        if file_exists:
            with open(filename, 'r', encoding='utf-8') as f:
                existing = f.read()
            if data["url"] in existing:
                print(f"スキップ（既存）：{data['product_name']}")
                return

        writer.writerow(data)
        print(f"追加保存：{data['product_name']}")


from urllib.parse import urljoin

def scrape_category_all_pages(base_url):
    all_links = set()
    visited_pages = set()
    next_pages = [base_url]

    while next_pages:
        url = next_pages.pop(0)
        if url in visited_pages:
            continue
        visited_pages.add(url)

        print(f"取得中: {url}")
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        # 商品リンク収集
        for a in soup.select("div.list_inner a"):
            href = a.get("href")
            if href and "/products/" in href:
                full_url = urljoin("https://www.sej.co.jp", href)
                all_links.add(full_url)

        # ページャーから次ページ候補を収集
        pager = soup.select("div.pager a")
        for a in pager:
            href = a.get("href")
            if href:
                next_url = urljoin("https://www.sej.co.jp", href)
                if next_url not in visited_pages:
                    next_pages.append(next_url)

    return list(all_links)



# 使用例
if __name__ == "__main__":
    category_url = "https://www.sej.co.jp/products/a/frozen_foods/"
    product_urls = scrape_category_all_pages(category_url)

    print(f"商品数: {len(product_urls)}")
    for url in product_urls:
        try:
            product_info = scrape_seven_eleven_product(url)
            save_to_csv(product_info)
        except Exception as e:
            print(f"エラー：{url} -> {e}")


取得中: https://www.sej.co.jp/products/a/frozen_foods/
商品数: 15
追加保存：７プレミアム塩味そら豆
追加保存：７プレミアム極上炒飯
追加保存：７プレミアムカリッと大学いも
追加保存：７プレミアム若鶏もも唐揚げ
追加保存：純氷３．５ｋｇ
追加保存：７プレミアムロックアイス１．１ｋｇ
追加保存：７プレミアムアップルマンゴー
追加保存：７プレミアム焼おにぎり
追加保存：７プレミアムブルーベリー
追加保存：７プレミアム炒飯（袋）
追加保存：７プレミアムカルビクッパ
追加保存：７プレミアム塩ゆで枝豆
追加保存：天然水の氷１．１Ｋｇ
追加保存：７プレミアムブロッコリー
追加保存：７プレミアムひとくち熟成蜜芋


In [23]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import os

# CSVファイル名
filename = "seven_eleven_product.csv"

# 栄養成分を正規表現で分解する関数
def parse_nutrition(text):
    nutrition = {
        "calorie": "N/A",
        "protein": "N/A",
        "fat": "N/A",
        "carbohydrate": "N/A",
        "sugar": "N/A",
        "fiber": "N/A",
        "salt": "N/A",
    }

    # 各項目を正規表現で抽出
    patterns = {
        "calorie": r"熱量[:：]\s*([\d\.]+kcal)",
        "protein": r"たんぱく質[:：]\s*([\d\.]+g)",
        "fat": r"脂質[:：]\s*([\d\.]+g)",
        "carbohydrate": r"炭水化物[:：]\s*([\d\.]+g)",
        "sugar": r"糖質[:：]\s*([\d\.]+g)",
        "fiber": r"食物繊維[:：]\s*([\d\.]+g)",
        "salt": r"食塩相当量[:：]\s*([\d\.]+g)",
    }

    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            nutrition[key] = match.group(1)

    return nutrition


# 商品ページから詳細を取得
def scrape_seven_eleven_product(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 商品名
    product_name_div = soup.find('div', class_='item_ttl')
    product_name_tag = product_name_div.find('h1') if product_name_div else None
    product_name = product_name_tag.get_text(strip=True).replace("\u3000", "") if product_name_tag else "N/A"

    # 価格
    price_tag = soup.find('div', class_='item_price')
    price = "N/A"
    if price_tag:
        price_text = price_tag.text.strip()
        price_match = re.search(r'(\d+円（税込\d+円）)', price_text)
        price = price_match.group(1) if price_match else price_text

    # 画像URL
    picture_url = ""
    product_wrap_div = soup.find('div', class_='productWrap')
    if product_wrap_div:
        img_tag = product_wrap_div.find('img')
        if img_tag and img_tag.has_attr("src"):
            picture_url = img_tag['src']

    # 栄養成分
    nutrition_td_text = ""
    for th in soup.find_all("th"):
        if "栄養成分" in th.get_text(strip=True):
            td = th.find_next("td")
            if td:
                nutrition_td_text = td.get_text(" ", strip=True)
                break
    nutrients = parse_nutrition(nutrition_td_text)

    return {
        "product_name": product_name,
        "price": price,
        "picture": picture_url,
        "url": url,
        **nutrients
    }


# CSVに保存（重複回避して追記）
def save_to_csv(data, filename=filename):
    file_exists = os.path.isfile(filename)

    fieldnames = ["product_name", "price", "picture", "url",
                  "calorie", "protein", "fat", "carbohydrate", "sugar", "fiber", "salt"]

    with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # 新規作成時のみヘッダーを書き込む
        if not file_exists:
            writer.writeheader()

        # 既存ファイルの重複チェック
        if file_exists:
            with open(filename, 'r', encoding='utf-8') as f:
                existing = f.read()
            if data["url"] in existing:
                print(f"スキップ（既存）：{data['product_name']}")
                return

        writer.writerow(data)
        print(f"追加保存：{data['product_name']}")


from urllib.parse import urljoin

def scrape_category_all_pages(base_url):
    all_links = set()
    visited_pages = set()
    next_pages = [base_url]

    while next_pages:
        url = next_pages.pop(0)
        if url in visited_pages:
            continue
        visited_pages.add(url)

        print(f"取得中: {url}")
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        # 商品リンク収集
        for a in soup.select("div.list_inner a"):
            href = a.get("href")
            if href and "/products/" in href:
                full_url = urljoin("https://www.sej.co.jp", href)
                all_links.add(full_url)

        # ページャーから次ページ候補を収集
        pager = soup.select("div.pager a")
        for a in pager:
            href = a.get("href")
            if href:
                next_url = urljoin("https://www.sej.co.jp", href)
                if next_url not in visited_pages:
                    next_pages.append(next_url)

    return list(all_links)



# 使用例
if __name__ == "__main__":
    category_url = "https://www.sej.co.jp/products/a/7premium/"
    product_urls = scrape_category_all_pages(category_url)

    print(f"商品数: {len(product_urls)}")
    for url in product_urls:
        try:
            product_info = scrape_seven_eleven_product(url)
            save_to_csv(product_info)
        except Exception as e:
            print(f"エラー：{url} -> {e}")


取得中: https://www.sej.co.jp/products/a/7premium/
商品数: 0


In [18]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import os

# CSVファイル名
filename = "seven_eleven_product.csv"

# 栄養成分を正規表現で分解する関数
def parse_nutrition(text):
    nutrition = {
        "calorie": "N/A",
        "protein": "N/A",
        "fat": "N/A",
        "carbohydrate": "N/A",
        "sugar": "N/A",
        "fiber": "N/A",
        "salt": "N/A",
    }

    # 各項目を正規表現で抽出
    patterns = {
        "calorie": r"熱量[:：]\s*([\d\.]+kcal)",
        "protein": r"たんぱく質[:：]\s*([\d\.]+g)",
        "fat": r"脂質[:：]\s*([\d\.]+g)",
        "carbohydrate": r"炭水化物[:：]\s*([\d\.]+g)",
        "sugar": r"糖質[:：]\s*([\d\.]+g)",
        "fiber": r"食物繊維[:：]\s*([\d\.]+g)",
        "salt": r"食塩相当量[:：]\s*([\d\.]+g)",
    }

    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            nutrition[key] = match.group(1)

    return nutrition


# 商品ページから詳細を取得
def scrape_seven_eleven_product(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 商品名
    product_name_div = soup.find('div', class_='item_ttl')
    product_name_tag = product_name_div.find('h1') if product_name_div else None
    product_name = product_name_tag.get_text(strip=True).replace("\u3000", "") if product_name_tag else "N/A"

    # 価格
    price_tag = soup.find('div', class_='item_price')
    price = "N/A"
    if price_tag:
        price_text = price_tag.text.strip()
        price_match = re.search(r'(\d+円（税込\d+円）)', price_text)
        price = price_match.group(1) if price_match else price_text

    # 画像URL
    picture_url = ""
    product_wrap_div = soup.find('div', class_='productWrap')
    if product_wrap_div:
        img_tag = product_wrap_div.find('img')
        if img_tag and img_tag.has_attr("src"):
            picture_url = img_tag['src']

    # 栄養成分
    nutrition_td_text = ""
    for th in soup.find_all("th"):
        if "栄養成分" in th.get_text(strip=True):
            td = th.find_next("td")
            if td:
                nutrition_td_text = td.get_text(" ", strip=True)
                break
    nutrients = parse_nutrition(nutrition_td_text)

    return {
        "product_name": product_name,
        "price": price,
        "picture": picture_url,
        "url": url,
        **nutrients
    }


# CSVに保存（重複回避して追記）
def save_to_csv(data, filename=filename):
    file_exists = os.path.isfile(filename)

    fieldnames = ["product_name", "price", "picture", "url",
                  "calorie", "protein", "fat", "carbohydrate", "sugar", "fiber", "salt"]

    with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # 新規作成時のみヘッダーを書き込む
        if not file_exists:
            writer.writeheader()

        # 既存ファイルの重複チェック
        if file_exists:
            with open(filename, 'r', encoding='utf-8') as f:
                existing = f.read()
            if data["url"] in existing:
                print(f"スキップ（既存）：{data['product_name']}")
                return

        writer.writerow(data)
        print(f"追加保存：{data['product_name']}")


from urllib.parse import urljoin

def scrape_category_all_pages(base_url):
    all_links = set()
    visited_pages = set()
    next_pages = [base_url]

    while next_pages:
        url = next_pages.pop(0)
        if url in visited_pages:
            continue
        visited_pages.add(url)

        print(f"取得中: {url}")
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        # 商品リンク収集
        for a in soup.select("div.list_inner a"):
            href = a.get("href")
            if href and "/products/" in href:
                full_url = urljoin("https://www.sej.co.jp", href)
                all_links.add(full_url)

        # ページャーから次ページ候補を収集
        pager = soup.select("div.pager a")
        for a in pager:
            href = a.get("href")
            if href:
                next_url = urljoin("https://www.sej.co.jp", href)
                if next_url not in visited_pages:
                    next_pages.append(next_url)

    return list(all_links)



# 使用例
if __name__ == "__main__":
    category_url = "https://www.sej.co.jp/products/a/sevencafe/"
    product_urls = scrape_category_all_pages(category_url)

    print(f"商品数: {len(product_urls)}")
    for url in product_urls:
        try:
            product_info = scrape_seven_eleven_product(url)
            save_to_csv(product_info)
        except Exception as e:
            print(f"エラー：{url} -> {e}")


取得中: https://www.sej.co.jp/products/a/sevencafe/
商品数: 8
追加保存：７カフェアイスカフェラテＬ
追加保存：セブンカフェホットコーヒーＬ
追加保存：セブンカフェアイスコーヒーＬ
追加保存：セブンカフェアイスコーヒーＲ
追加保存：７カフェアイスカフェラテＲ
追加保存：７カフェホットカフェラテＲ
追加保存：７カフェホットカフェラテＬ
追加保存：セブンカフェホットコーヒーＲ


In [21]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import os

# CSVファイル名
filename = "seven_eleven_product.csv"

# 栄養成分を正規表現で分解する関数
def parse_nutrition(text):
    nutrition = {
        "calorie": "N/A",
        "protein": "N/A",
        "fat": "N/A",
        "carbohydrate": "N/A",
        "sugar": "N/A",
        "fiber": "N/A",
        "salt": "N/A",
    }

    # 各項目を正規表現で抽出
    patterns = {
        "calorie": r"熱量[:：]\s*([\d\.]+kcal)",
        "protein": r"たんぱく質[:：]\s*([\d\.]+g)",
        "fat": r"脂質[:：]\s*([\d\.]+g)",
        "carbohydrate": r"炭水化物[:：]\s*([\d\.]+g)",
        "sugar": r"糖質[:：]\s*([\d\.]+g)",
        "fiber": r"食物繊維[:：]\s*([\d\.]+g)",
        "salt": r"食塩相当量[:：]\s*([\d\.]+g)",
    }

    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            nutrition[key] = match.group(1)

    return nutrition


# 商品ページから詳細を取得
def scrape_seven_eleven_product(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 商品名
    product_name_div = soup.find('div', class_='item_ttl')
    product_name_tag = product_name_div.find('h1') if product_name_div else None
    product_name = product_name_tag.get_text(strip=True).replace("\u3000", "") if product_name_tag else "N/A"

    # 価格
    price_tag = soup.find('div', class_='item_price')
    price = "N/A"
    if price_tag:
        price_text = price_tag.text.strip()
        price_match = re.search(r'(\d+円（税込\d+円）)', price_text)
        price = price_match.group(1) if price_match else price_text

    # 画像URL
    picture_url = ""
    product_wrap_div = soup.find('div', class_='productWrap')
    if product_wrap_div:
        img_tag = product_wrap_div.find('img')
        if img_tag and img_tag.has_attr("src"):
            picture_url = img_tag['src']

    # 栄養成分
    nutrition_td_text = ""
    for th in soup.find_all("th"):
        if "栄養成分" in th.get_text(strip=True):
            td = th.find_next("td")
            if td:
                nutrition_td_text = td.get_text(" ", strip=True)
                break
    nutrients = parse_nutrition(nutrition_td_text)

    return {
        "product_name": product_name,
        "price": price,
        "picture": picture_url,
        "url": url,
        **nutrients
    }


# CSVに保存（重複回避して追記）
def save_to_csv(data, filename=filename):
    file_exists = os.path.isfile(filename)

    fieldnames = ["product_name", "price", "picture", "url",
                  "calorie", "protein", "fat", "carbohydrate", "sugar", "fiber", "salt"]

    with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # 新規作成時のみヘッダーを書き込む
        if not file_exists:
            writer.writeheader()

        # 既存ファイルの重複チェック
        if file_exists:
            with open(filename, 'r', encoding='utf-8') as f:
                existing = f.read()
            if data["url"] in existing:
                print(f"スキップ（既存）：{data['product_name']}")
                return

        writer.writerow(data)
        print(f"追加保存：{data['product_name']}")


from urllib.parse import urljoin

def scrape_category_all_pages(base_url):
    all_links = set()
    visited_pages = set()
    next_pages = [base_url]

    while next_pages:
        url = next_pages.pop(0)
        if url in visited_pages:
            continue
        visited_pages.add(url)

        print(f"取得中: {url}")
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        # 商品リンク収集
        for a in soup.select("div.list_inner a"):
            href = a.get("href")
            if href and "/products/" in href:
                full_url = urljoin("https://www.sej.co.jp", href)
                all_links.add(full_url)

        # ページャーから次ページ候補を収集
        pager = soup.select("div.pager a")
        for a in pager:
            href = a.get("href")
            if href:
                next_url = urljoin("https://www.sej.co.jp", href)
                if next_url not in visited_pages:
                    next_pages.append(next_url)

    return list(all_links)



# 使用例
if __name__ == "__main__":
    category_url = "https://www.sej.co.jp/products/sevencafebakery/"
    product_urls = scrape_category_all_pages(category_url)

    print(f"商品数: {len(product_urls)}")
    for url in product_urls:
        try:
            product_info = scrape_seven_eleven_product(url)
            save_to_csv(product_info)
        except Exception as e:
            print(f"エラー：{url} -> {e}")


取得中: https://www.sej.co.jp/products/sevencafebakery/
商品数: 0


In [4]:
import time
import requests
from bs4 import BeautifulSoup
import csv
import re
import os
from urllib.parse import urljoin

# CSVファイル名
filename = "seven_eleven_products.csv"

# 栄養成分を正規表現で分解する関数
def parse_nutrition(text):
    nutrition = {
        "calorie": "N/A",
        "protein": "N/A",
        "fat": "N/A",
        "carbohydrate": "N/A",
        "sugar": "N/A",
        "fiber": "N/A",
        "salt": "N/A",
    }

    patterns = {
        "calorie": r"熱量[:：]\s*([\d\.]+kcal)",
        "protein": r"たんぱく質[:：]\s*([\d\.]+g)",
        "fat": r"脂質[:：]\s*([\d\.]+g)",
        "carbohydrate": r"炭水化物[:：]\s*([\d\.]+g)",
        "sugar": r"糖質[:：]\s*([\d\.]+g)",
        "fiber": r"食物繊維[:：]\s*([\d\.]+g)",
        "salt": r"食塩相当量[:：]\s*([\d\.]+g)",
    }

    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            nutrition[key] = match.group(1)

    return nutrition


# 商品ページから詳細を取得
def scrape_seven_eleven_product(url, category_name):
    response = requests.get(url)
    response.encoding = response.apparent_encoding
    soup = BeautifulSoup(response.text, 'html.parser')

    # 商品名
    product_name = "N/A"
    product_name_div = soup.find('div', class_='item_ttl')
    if product_name_div:
        h1 = product_name_div.find('h1')
        if h1:
            product_name = h1.get_text(strip=True).replace("\u3000", "")

    # 価格
    price = "N/A"
    price_div = soup.find('div', class_='item_price')
    if price_div:
        price_text = price_div.get_text(strip=True)
        match = re.search(r'(\d+円（税込[\d\.]+円）)', price_text)
        if match:
            price = match.group(1)
        else:
            price = price_text

    # 画像URL
    picture_url = "N/A"
    product_wrap_div = soup.find('div', class_='productWrap')
    if product_wrap_div:
        img = product_wrap_div.find('img')
        if img and img.has_attr("src"):
            picture_url = img["src"]

    # 栄養成分
    nutrition_text = ""
    for th in soup.find_all("th"):
        if "栄養成分" in th.get_text(strip=True):
            td = th.find_next("td")
            if td:
                nutrition_text = td.get_text(" ", strip=True)
                break
    nutrients = parse_nutrition(nutrition_text)

    return {
        "category": category_name,
        "product_name": product_name,
        "price": price,
        "picture": picture_url,
        "url": url,
        **nutrients
    }


# CSVに保存（重複回避付き）
def save_to_csv(data, filename=filename):
    file_exists = os.path.isfile(filename)
    fieldnames = [
        "category", "product_name", "price", "picture", "url",
        "calorie", "protein", "fat", "carbohydrate", "sugar", "fiber", "salt"
    ]

    # 既存URLを読み込み
    existing_urls = set()
    if file_exists:
        with open(filename, "r", encoding="utf-8") as f:
            reader = csv.DictReader(f)
            for row in reader:
                existing_urls.add(row["url"])

    if data["url"] in existing_urls:
        print(f"スキップ（既存）：{data['product_name']}")
        return

    with open(filename, "a", newline="", encoding="utf-8") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()
        writer.writerow(data)
        print(f"追加保存：{data['product_name']}")


# トップページからカテゴリ一覧を取得
def scrape_all_category_urls():
    base_url = "https://www.sej.co.jp/products"
    response = requests.get(base_url)
    response.encoding = response.apparent_encoding
    soup = BeautifulSoup(response.text, "html.parser")

    categories = []
    for li in soup.select("ul.link-img-list li"):
        a = li.select_one("a ")
        name = li.select_one("figcaption p.ttl")
        if a and name:
            url = urljoin(base_url, a["href"])
            categories.append((name.get_text(" ", strip=True), url))

    return categories


# カテゴリページをクロールして全ページの商品URLを収集
def scrape_category_all_pages(base_url):
    all_links = set()
    visited_pages = set()
    next_pages = [base_url]

    while next_pages:
        url = next_pages.pop(0)
        if url in visited_pages:
            continue
        visited_pages.add(url)

        print(f"取得中: {url}")
        response = requests.get(url)
        response.encoding = response.apparent_encoding
        soup = BeautifulSoup(response.text, "html.parser")

        # 商品リンク
        for a in soup.select("div.list_inner"):
            href = a.get("a href")
            if href and "/products/" in href:
                full_url = urljoin("https://www.sej.co.jp", href)
                all_links.add(full_url)

        # ページャー
        for a in soup.select("div.pager"):
            href = a.get("a href")
            if href:
                next_url = urljoin("https://www.sej.co.jp", href)
                if next_url not in visited_pages:
                    next_pages.append(next_url)

        time.sleep(1)  # ページ間の待機

    return list(all_links)


# メイン処理
if __name__ == "__main__":
    categories = scrape_all_category_urls()
    print(f"カテゴリ数: {len(categories)}")

    for category_name, category_url in categories:
        print(f"\n--- カテゴリ開始: {category_name} ---")
        product_urls = scrape_category_all_pages(category_url)
        print(f"商品URL数: {len(product_urls)}")

        for url in product_urls:
            try:
                product_info = scrape_seven_eleven_product(url, category_name)
                save_to_csv(product_info)
                time.sleep(1)  # 商品ごとに待機
            except Exception as e:
                print(f"エラー：{url} -> {e}")

カテゴリ数: 40

--- カテゴリ開始: おにぎり ---
取得中: https://www.sej.co.jp/products/a/onigiri/
商品URL数: 0

--- カテゴリ開始: お寿司 ---
取得中: https://www.sej.co.jp/products/a/sushi/
商品URL数: 0

--- カテゴリ開始: お弁当 ---
取得中: https://www.sej.co.jp/products/a/bento/
商品URL数: 0

--- カテゴリ開始: サンドイッチ・ ロールパン ---
取得中: https://www.sej.co.jp/products/a/sandwich/
商品URL数: 0

--- カテゴリ開始: パン ---
取得中: https://www.sej.co.jp/products/a/bread/


KeyboardInterrupt: 

In [7]:
import time
import requests
from bs4 import BeautifulSoup
import csv
import re
import os
from urllib.parse import urljoin

# CSVファイル名
filename = "seven_eleven_products.csv"

# 栄養成分を正規表現で分解する関数
def parse_nutrition(text):
    nutrition = {
        "calorie": "N/A",
        "protein": "N/A",
        "fat": "N/A",
        "carbohydrate": "N/A",
        "sugar": "N/A",
        "fiber": "N/A",
        "salt": "N/A",
    }

    patterns = {
        "calorie": r"熱量[:：]\s*([\d\.]+kcal)",
        "protein": r"たんぱく質[:：]\s*([\d\.]+g)",
        "fat": r"脂質[:：]\s*([\d\.]+g)",
        "carbohydrate": r"炭水化物[:：]\s*([\d\.]+g)",
        "sugar": r"糖質[:：]\s*([\d\.]+g)",
        "fiber": r"食物繊維[:：]\s*([\d\.]+g)",
        "salt": r"食塩相当量[:：]\s*([\d\.]+g)",
    }

    for key, pattern in patterns.items():
        match = re.search(pattern, text)
        if match:
            nutrition[key] = match.group(1)

    return nutrition

# 商品ページから詳細を取得
def scrape_seven_eleven_product(url, category_name):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    # 商品名
    product_name_div = soup.find('div', class_='item_ttl')
    product_name_tag = product_name_div.find('h1') if product_name_div else None
    product_name = product_name_tag.get_text(strip=True).replace("\u3000", "") if product_name_tag else "N/A"

    # 価格
    price_tag = soup.find('div', class_='item_price')
    price = "N/A"
    if price_tag:
        price_text = price_tag.text.strip()
        price_match = re.search(r'(\d+円（税込[\d\.]+円）)', price_text)
        price = price_match.group(1) if price_match else price_text

    # 画像URL
    picture_url = "N/A"
    product_wrap_div = soup.find('div', class_='productWrap')
    if product_wrap_div:
        img_tag = product_wrap_div.find('img')
        if img_tag and img_tag.has_attr("src"):
            picture_url = img_tag['src']

    # 栄養成分
    nutrition_td_text = ""
    for th in soup.find_all("th"):
        if "栄養成分" in th.get_text(strip=True):
            td = th.find_next("td")
            if td:
                nutrition_td_text = td.get_text(" ", strip=True)
                break
    nutrients = parse_nutrition(nutrition_td_text)

    return {
        "category": category_name,
        "product_name": product_name or "N/A",
        "price": price or "N/A",
        "picture": picture_url or "N/A",
        "url": url,
        **nutrients
    }

# CSVに保存（重複回避付き）
def save_to_csv(data, filename=filename):
    if not data or "url" not in data:
        print(f"スキップ（データ不正）: {data}")
        return

    file_exists = os.path.isfile(filename)
    fieldnames = ["category","product_name", "price", "picture", "url",
                  "calorie", "protein", "fat", "carbohydrate", "sugar", "fiber", "salt"]

    # 既存URL読み込み
    existing_urls = set()
    if file_exists:
        with open(filename, 'r', encoding='utf-8') as f:
            reader = csv.DictReader(f)
            for row in reader:
                existing_urls.add(row["url"])

    if data["url"] in existing_urls:
        print(f"スキップ（既存）：{data['product_name']}")
        return

    with open(filename, 'a', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        if not file_exists:
            writer.writeheader()
        writer.writerow(data)
        print(f"追加保存：{data['product_name']}")

# 指定のdivだけからカテゴリを取得
def scrape_specific_category():
    base_url = "https://www.sej.co.jp/products"
    response = requests.get(base_url)
    response.encoding = response.apparent_encoding
    soup = BeautifulSoup(response.text, "html.parser")

    target_div = soup.find("div", id="pbBlock3329309")
    categories = []

    if target_div:
        for li in target_div.select("ul.link-img-list li"):
            a = li.select_one("a")
            name = li.select_one("figcaption p.ttl")
            if a and name and a.has_attr("href"):
                url = urljoin(base_url, a["href"])
                categories.append((name.get_text(" ", strip=True), url))
    return categories

# カテゴリページの全商品URLを取得（全ページ対応）
def scrape_category_all_pages(base_url):
    all_links = set()
    visited_pages = set()
    next_pages = [base_url]

    while next_pages:
        url = next_pages.pop(0)
        if url in visited_pages:
            continue
        visited_pages.add(url)

        print(f"取得中: {url}")
        response = requests.get(url)
        soup = BeautifulSoup(response.content, "html.parser")

        # 商品リンク収集
        for div in soup.select("div.list_inner"):
            a = div.find("a")
            if a and a.has_attr("href") and "/products/" in a["href"]:
                full_url = urljoin("https://www.sej.co.jp", a["href"])
                all_links.add(full_url)

        # ページャー
        for a in soup.select("div.pager a"):
            if a.has_attr("href"):
                next_url = urljoin("https://www.sej.co.jp", a["href"])
                if next_url not in visited_pages:
                    next_pages.append(next_url)

        time.sleep(1)

    return list(all_links)

# メイン処理
if __name__ == "__main__":
    categories = scrape_specific_category()
    print(f"カテゴリ数: {len(categories)}")

    for category_name, category_url in categories:
        print(f"\n--- カテゴリ開始: {category_name} ---")
        product_urls = scrape_category_all_pages(category_url)
        print(f"商品URL数: {len(product_urls)}")

        for url in product_urls:
            try:
                product_info = scrape_seven_eleven_product(url, category_name)
                save_to_csv(product_info)
                time.sleep(1)
            except Exception as e:
                print(f"エラー：{url} -> {e}")


カテゴリ数: 20

--- カテゴリ開始: おにぎり ---
取得中: https://www.sej.co.jp/products/a/onigiri/
商品URL数: 18
エラー：https://www.sej.co.jp/products/a/item/047791/ -> 'url'
エラー：https://www.sej.co.jp/products/a/item/044395/ -> 'url'
エラー：https://www.sej.co.jp/products/a/item/047813/ -> 'url'
エラー：https://www.sej.co.jp/products/a/item/047906/ -> 'url'
エラー：https://www.sej.co.jp/products/a/item/047794/ -> 'url'
エラー：https://www.sej.co.jp/products/a/item/047776/ -> 'url'
エラー：https://www.sej.co.jp/products/a/item/044425/ -> 'url'
エラー：https://www.sej.co.jp/products/a/item/047792/ -> 'url'
エラー：https://www.sej.co.jp/products/a/item/047940/ -> 'url'
エラー：https://www.sej.co.jp/products/a/item/047801/ -> 'url'
エラー：https://www.sej.co.jp/products/a/item/047626/ -> 'url'
エラー：https://www.sej.co.jp/products/a/item/047809/ -> 'url'
エラー：https://www.sej.co.jp/products/a/item/047683/ -> 'url'
エラー：https://www.sej.co.jp/products/a/item/047939/ -> 'url'
エラー：https://www.sej.co.jp/products/a/item/045492/ -> 'url'
エラー：https://www.sej.co.jp

KeyboardInterrupt: 

In [9]:
import requests
from bs4 import BeautifulSoup
import csv
import time
import re
from urllib.parse import urljoin, urlparse
import json

class SejSaladScraper:
    def __init__(self):
        self.base_url = "https://www.sej.co.jp"
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8',
            'Accept-Language': 'ja-JP,ja;q=0.9,en-US;q=0.8,en;q=0.7',
            'Accept-Encoding': 'gzip, deflate, br',
            'Connection': 'keep-alive',
            'Upgrade-Insecure-Requests': '1',
            'Sec-Fetch-Dest': 'document',
            'Sec-Fetch-Mode': 'navigate',
            'Sec-Fetch-Site': 'none',
            'Cache-Control': 'max-age=0',
        })
    
    def get_products_from_page(self, page_url):
        """指定されたページから商品一覧を取得"""
        try:
            response = self.session.get(page_url)
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            
            products = []
            
            # 参考コードのHTML構造に合わせたセレクタを使用
            # div class="list_inner -item-code-XXXXXX"
            product_elements = soup.select('div[class*="list_inner -item-code-"]')
            
            if not product_elements:
                print(f"警告: 商品要素が見つかりませんでした。セレクタ '.list_inner -item-code-*' を確認してください。")
                return []

            print(f"商品要素を発見: {len(product_elements)}件")
            
            for element in product_elements:
                try:
                    # 商品名を取得
                    # <div class="item_ttl"><p><a href="...">７Ｐサラダチキンバー　バジル＆オリーブ</a></p></div>
                    product_name_element = element.select_one('.item_ttl p a')
                    product_name = product_name_element.get_text(strip=True) if product_name_element else None
                    
                    # 「サラダ」が含まれる商品のみを対象にする
                    if product_name and 'サラダ' in product_name:
                        # 商品詳細ページのURLを取得
                        # <figure><a href="/products/a/item/251383/"><img ...></a></figure>
                        product_url_element = element.select_one('figure a')
                        product_url = urljoin(self.base_url, product_url_element['href']) if product_url_element and 'href' in product_url_element.attrs else None
                        
                        # 価格を取得
                        # <div class="item_price"><p>148円（税込159.84円）</p></div>
                        price = None
                        price_element = element.select_one('.item_price p')
                        if price_element:
                            price_text = price_element.get_text(strip=True)
                            price_match = re.search(r'(\d+(?:,\d+)?)', price_text.replace(',', ''))
                            if price_match:
                                price = price_match.group(1)
                        
                        # 画像を取得
                        # <img data-original="..." alt="" class="lazy" src="...">
                        picture = None
                        img_element = element.select_one('figure img')
                        if img_element:
                            # data-original属性があればそちらを優先、なければsrc
                            picture = urljoin(self.base_url, img_element.get('data-original', img_element.get('src', '')))
                        
                        products.append({
                            'product_name': product_name,
                            'price': price,
                            'picture': picture,
                            'url': product_url,
                            'source_page': page_url
                        })
                        
                        print(f"サラダ商品を発見: {product_name}")
                
                except Exception as e:
                    print(f"商品情報取得エラー (要素: {element.prettify()[:200]}...): {e}") # エラー発生箇所のHTMLを一部表示
                    continue
            
            return products
        
        except Exception as e:
            print(f"ページ取得エラー ({page_url}): {e}")
            return []
    
    def extract_nutrition_info(self, soup):
        """栄養成分情報を抽出"""
        nutrition_data = {
            'calorie': None,
            'protein': None,
            'fat': None,
            'carbohydrate': None,
            'sugar': None,
            'fiber': None,
            'salt': None
        }
        
        # 栄養成分表を探す
        nutrition_sections = []
        
        # テーブル形式
        tables = soup.find_all('table')
        for table in tables:
            table_text = table.get_text()
            if any(keyword in table_text for keyword in ['エネルギー', 'カロリー', 'たんぱく質', '栄養成分', '栄養']):
                nutrition_sections.append(table)
        
        # div要素
        nutrition_divs = soup.find_all('div', string=re.compile(r'栄養成分|エネルギー|カロリー|栄養'))
        for div in nutrition_divs:
            parent = div.find_parent()
            if parent:
                nutrition_sections.append(parent)
        
        # クラス名で探す
        # セブンイレブンの商品詳細ページでよく見られるセレクタを追加
        nutrition_classes = [
            '.item-nutritional-info', # 例: <div class="item-nutritional-info">
            '.product-info-table',    # 例: <table class="product-info-table">
            '.nutrition-facts',       # 一般的なクラス名
            '.nutrition',
            '.nutritional-info',
            '.product-info',
            '.detail-info'
        ]
        
        for class_name in nutrition_classes:
            elements = soup.select(class_name) # selectを使用
            nutrition_sections.extend(elements)

        # 重複する要素を排除
        nutrition_sections = list(set(nutrition_sections))
        
        # 栄養成分を抽出
        for section in nutrition_sections:
            if section:
                text = section.get_text(separator=' ', strip=True) # 改行や複数の空白を整理
                
                patterns = {
                    'calorie': [
                        r'エネルギー[：:\s]*(\d+(?:\.\d+)?)\s*(?:kcal|キロカロリー)?',
                        r'カロリー[：:\s]*(\d+(?:\.\d+)?)\s*(?:kcal)?',
                        r'(\d+(?:\.\d+)?)\s*kcal'
                    ],
                    'protein': [
                        r'たんぱく質[：:\s]*(\d+(?:\.\d+)?)\s*g?',
                        r'蛋白質[：:\s]*(\d+(?:\.\d+)?)\s*g?',
                        r'タンパク質[：:\s]*(\d+(?:\.\d+)?)\s*g?'
                    ],
                    'fat': [
                        r'脂質[：:\s]*(\d+(?:\.\d+)?)\s*g?',
                        r'脂肪[：:\s]*(\d+(?:\.\d+)?)\s*g?'
                    ],
                    'carbohydrate': [
                        r'炭水化物[：:\s]*(\d+(?:\.\d+)?)\s*g?'
                    ],
                    'sugar': [
                        r'糖質[：:\s]*(\d+(?:\.\d+)?)\s*g?',
                        r'糖分[：:\s]*(\d+(?:\.\d+)?)\s*g?'
                    ],
                    'fiber': [
                        r'食物繊維[：:\s]*(\d+(?:\.\d+)?)\s*g?',
                        r'繊維[：:\s]*(\d+(?:\.\d+)?)\s*g?'
                    ],
                    'salt': [
                        r'食塩相当量[：:\s]*(\d+(?:\.\d+)?)\s*g?',
                        r'塩分[：:\s]*(\d+(?:\.\d+)?)\s*g?',
                        r'ナトリウム[：:\s]*(\d+(?:\.\d+)?)mg?', # ナトリウムの場合はmgも考慮
                        r'食塩[：:\s]*(\d+(?:\.\d+)?)\s*g?'
                    ]
                }
                
                for nutrient, regex_list in patterns.items():
                    if nutrition_data[nutrient] is None:
                        for regex in regex_list:
                            match = re.search(regex, text, re.IGNORECASE)
                            if match:
                                value = match.group(1)
                                # ナトリウムを食塩相当量に変換する場合のロジックを追加（任意）
                                if nutrient == 'salt' and 'ナトリウム' in regex and 'mg' in text:
                                    try:
                                        # ナトリウム(mg) * 2.54 / 1000 = 食塩相当量(g)
                                        # ただし、すでに食塩相当量として表示されている場合もあるので、優先順位は調整
                                        sodium_mg = float(value)
                                        value = str(round(sodium_mg * 2.54 / 1000, 2))
                                    except ValueError:
                                        pass # 変換失敗時はそのまま
                                nutrition_data[nutrient] = value
                                break
        
        return nutrition_data
    
    def scrape_product_detail(self, product_info):
        """商品詳細ページから栄養成分情報を取得"""
        try:
            if not product_info.get('url'):
                print(f"  詳細URL無し: {product_info.get('product_name', 'Unknown')}")
                return {**product_info, **{
                    'calorie': None, 'protein': None, 'fat': None,
                    'carbohydrate': None, 'sugar': None, 'fiber': None, 'salt': None
                }}
            
            response = self.session.get(product_info['url'])
            response.raise_for_status()
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # 栄養成分情報を取得
            nutrition_info = self.extract_nutrition_info(soup)
            
            # 栄養成分が取得できなかった場合に、別のセレクタを試すなど、さらに頑健性を高めることも可能
            
            return {**product_info, **nutrition_info}
        
        except requests.exceptions.HTTPError as http_err:
            print(f"  HTTPエラー ({product_info.get('product_name', 'Unknown')}, URL: {product_info.get('url')}): {http_err}")
            return {**product_info, **{
                'calorie': None, 'protein': None, 'fat': None,
                'carbohydrate': None, 'sugar': None, 'fiber': None, 'salt': None
            }}
        except Exception as e:
            print(f"  商品詳細取得エラー ({product_info.get('product_name', 'Unknown')}, URL: {product_info.get('url')}): {e}")
            return {**product_info, **{
                'calorie': None, 'protein': None, 'fat': None,
                'carbohydrate': None, 'sugar': None, 'fiber': None, 'salt': None
            }}
    
    def scrape_multiple_pages(self, page_urls, delay=2):
        """複数ページから全商品の情報を取得"""
        all_products = []
        
        for i, page_url in enumerate(page_urls, 1):
            print(f"\n=== ページ {i}/{len(page_urls)} を処理中 ===")
            print(f"URL: {page_url}")
            
            # ページから商品一覧を取得
            products_basic_info = self.get_products_from_page(page_url)
            print(f"サラダ商品を {len(products_basic_info)} 件発見")
            
            # 各商品の詳細情報を取得
            for j, product_info in enumerate(products_basic_info, 1):
                print(f"  商品 {j}/{len(products_basic_info)}: {product_info.get('product_name', 'N/A')}")
                
                complete_product_data = self.scrape_product_detail(product_info)
                all_products.append(complete_product_data)
                
                # サーバーへの負荷を軽減
                if j < len(products_basic_info):
                    time.sleep(delay)
            
            # ページ間の遅延
            if i < len(page_urls):
                time.sleep(delay * 2)
        
        return all_products
    
    def save_to_csv(self, data, filename='sej_salad_products.csv'):
        """データをCSVファイルに保存"""
        if not data:
            print("保存するデータがありません。")
            return False
        
        columns = ["product_name", "price", "picture", "calorie", "protein", "fat", 
        "carbohydrate", "sugar", "fiber", "salt", "url"]
        
        try:
            with open(filename, 'w', newline='', encoding='utf-8-sig') as csvfile:
                writer = csv.DictWriter(csvfile, fieldnames=columns)
                writer.writeheader()
                
                for row in data:
                    filtered_row = {col: row.get(col, '') for col in columns}
                    writer.writerow(filtered_row)
            
            print(f"\nデータを {filename} に保存しました。")
            print(f"保存された行数: {len(data)}")
            
            return True
            
        except Exception as e:
            print(f"CSV保存エラー: {e}")
            return False

# 使用例
def main():
    scraper = SejSaladScraper()
    
    # 指定されたURLリスト
    # セブンイレブンの「セブンプレミアム フレッシュ」カテゴリページや、サラダチキンなどがまとまっているページ
    page_urls = [
        "https://www.sej.co.jp/products/a/7premium/fresh/1/l100/" # フレッシュ惣菜など
    ]
    
    print("=== セブンイレブン サラダ商品スクレイピング開始 ===")
    print(f"対象ページ数: {len(page_urls)}")
    
    # データを取得
    products_data = scraper.scrape_multiple_pages(page_urls, delay=1.5)
    
    if products_data:
        # CSVに保存
        success = scraper.save_to_csv(products_data)
        
        if success:
            print("\n=== 取得データ概要 ===")
            print(f"総サラダ商品数: {len(products_data)}")
            
            # データ項目の取得状況
            data_status = {}
            for key in ['product_name', 'price', 'picture', 'calorie', 'protein', 'fat', 'carbohydrate', 'sugar', 'fiber', 'salt']:
                non_null_count = sum(1 for product in products_data if product.get(key) is not None and product.get(key) != '') # Noneと空文字をチェック
                data_status[key] = f"{non_null_count}/{len(products_data)}"
            
            print("\n各データ項目の取得状況:")
            for key, status in data_status.items():
                print(f"  {key}: {status}")
            
            # サンプルデータを表示
            print("\n=== サンプルデータ ===")
            for i, product in enumerate(products_data[:3], 1): # 最初の3件を表示
                print(f"\n商品 {i}:")
                print(f"  商品名: {product.get('product_name', 'N/A')}")
                print(f"  価格: {product.get('price', 'N/A')}円")
                print(f"  カロリー: {product.get('calorie', 'N/A')}kcal")
                print(f"  たんぱく質: {product.get('protein', 'N/A')}g")
                print(f"  脂質: {product.get('fat', 'N/A')}g")
                print(f"  炭水化物: {product.get('carbohydrate', 'N/A')}g")
                print(f"  糖質: {product.get('sugar', 'N/A')}g")
                print(f"  食物繊維: {product.get('fiber', 'N/A')}g")
                print(f"  食塩相当量: {product.get('salt', 'N/A')}g")
                print(f"  商品URL: {product.get('url', 'N/A')}")
                print(f"  取得元ページ: {product.get('source_page', 'N/A')}")
        
    else:
        print("サラダ商品が見つかりませんでした。")
        print("指定したURLやページ構造が変更されている可能性があります。")

if __name__ == "__main__":
    main()

=== セブンイレブン サラダ商品スクレイピング開始 ===
対象ページ数: 1

=== ページ 1/1 を処理中 ===
URL: https://www.sej.co.jp/products/a/7premium/fresh/1/l100/
商品要素を発見: 100件
サラダ商品を発見: ７Ｐサラダチキンバー　バジル＆オリーブ
サラダ商品を発見: ７Ｐサラダチキンバー　スモークペッパー
サラダ商品を発見: ７プレミアム　サラダチキン　スモーク
サラダ商品を発見: ７プレミアム　サラダチキン　ハーブ
サラダ商品を発見: ７プレミアム　　　　　サラダチキンバー
サラダ商品を発見: ７プレミアム　サラダチキン　プレーン
サラダ商品を発見: ７プレミアム　ほぐしサラダチキン
サラダ商品を 7 件発見
  商品 1/7: ７Ｐサラダチキンバー　バジル＆オリーブ
  商品 2/7: ７Ｐサラダチキンバー　スモークペッパー
  商品 3/7: ７プレミアム　サラダチキン　スモーク
  商品 4/7: ７プレミアム　サラダチキン　ハーブ
  商品 5/7: ７プレミアム　　　　　サラダチキンバー
  商品 6/7: ７プレミアム　サラダチキン　プレーン
  商品 7/7: ７プレミアム　ほぐしサラダチキン

データを sej_salad_products.csv に保存しました。
保存された行数: 7

=== 取得データ概要 ===
総サラダ商品数: 7

各データ項目の取得状況:
  product_name: 7/7
  price: 7/7
  picture: 7/7
  calorie: 7/7
  protein: 7/7
  fat: 6/7
  carbohydrate: 7/7
  sugar: 7/7
  fiber: 7/7
  salt: 7/7

=== サンプルデータ ===

商品 1:
  商品名: ７Ｐサラダチキンバー　バジル＆オリーブ
  価格: 148円
  カロリー: 59kcal
  たんぱく質: 12.5g
  脂質: 0.9g
  炭水化物: 0.0g
  糖質: 0.0g
  食物繊維: 0.0g
  食塩相当量: 0.8g
  商品URL: https://www.sej.co.jp/products/a/item/2513